# Working with async

`openaivec.aio` is a experimental module that provides an async interface to OpenAI's API. It is designed to be used with Python's `asyncio` library, allowing you to make non-blocking requests to the OpenAI API.

In [1]:
import os
from openai import AsyncOpenAI
from openaivec.aio import pandas_ext
import pandas as pd
from pydantic import BaseModel, Field
from enum import Enum

pandas_ext.use(AsyncOpenAI())
pandas_ext.responses_model("gpt-4.1-mini")
pandas_ext.embeddings_model("text-embedding-3-small")

The model name 'gpt-4.1-mini' is not supported by tiktoken. Instead, using the 'o200k_base' encoding.


In [2]:
files_dict = {"path": []}
for root, dirs, files in os.walk('../../src/openaivec'):
    for file in files:
        path = os.path.join(root, file)
        if "__" not in path:
            files_dict["path"].append(os.path.join(root, file))

implements_df = pd.DataFrame(files_dict).assign(
    module=lambda df: df["path"].str.split("/")
        .map(lambda x: x[3:])
        .map(lambda x: ".".join(x))
        .map(lambda x: x.replace(".py", "")),
)

In [3]:
implements_df

,path,module
0,../../src/openaivec/serialize.py,openaivec.serialize
1,../../src/openaivec/responses.py,openaivec.responses
2,../../src/openaivec/log.py,openaivec.log
3,../../src/openaivec/util.py,openaivec.util
4,../../src/openaivec/embeddings.py,openaivec.embeddings
5,../../src/openaivec/pandas_ext.py,openaivec.pandas_ext
6,../../src/openaivec/spark.py,openaivec.spark
7,../../src/openaivec/prompt.py,openaivec.prompt
8,../../src/openaivec/beta/batch.py,openaivec.beta.batch
9,../../src/openaivec/beta/schema.py,openaivec.beta.schema


In [4]:
class OpjectType(str, Enum):
    FUNCTION = "function"
    CLASS = "class"
    VARIABLE = "variable"

class Question(BaseModel):
    question: str = Field(description="The specific question related to the code section.")
    answer: str = Field(description="The corresponding answer explaining the code aspect.")

class Section(BaseModel):
    name: str = Field(description="The name of the function or class being documented.")
    type: OpjectType = Field(description="The type of the code section, either a function or a class.")
    description: str = Field(description="A concise summary of the function or class's purpose and functionality.")
    questions: list[Question] = Field(description="A list of Q&A pairs clarifying aspects of this code section.")

class Document(BaseModel):
    sections: list[Section] = Field(description="A list of sections, each documenting a specific function or class.")

Note that async methods are not available in lambda functions.

In [5]:
codes_df = implements_df.assign(
        code=lambda df: df['path'].apply(lambda x: open(x).read()),
)

docs_df = codes_df.assign(
    doc=await codes_df["code"].aio.responses(
        instructions="Generate a detailed documentation for the code, including sections for each function or class. Each section should include a name, description, and a list of questions and answers that clarify the code's functionality.",
        response_format=Document,
        batch_size=1,
    )
).pipe(
    lambda df: df.drop(columns=["code", "path"], errors="ignore")
)


In [6]:
docs_df

,module,doc
0,openaivec.serialize,"sections=[Section(name='serialize_base_model',..."
1,openaivec.responses,sections=[Section(name='_vectorize_system_mess...
2,openaivec.log,"sections=[Section(name='observe', type=<Opject..."
3,openaivec.util,"sections=[Section(name='split_to_minibatch', t..."
4,openaivec.embeddings,"sections=[Section(name='VectorizedEmbeddings',..."
5,openaivec.pandas_ext,"sections=[Section(name='use', type=<OpjectType..."
6,openaivec.spark,"sections=[Section(name='UDFBuilder', type=<Opj..."
7,openaivec.prompt,"sections=[Section(name='Example', type=<Opject..."
8,openaivec.beta.batch,"sections=[Section(name='JsonlUDFBuilder', type..."
9,openaivec.beta.schema,"sections=[Section(name='Entity', type=<OpjectT..."


In [7]:
questions_df = docs_df.pipe(
    lambda df: df
    .aio.extract("doc")
    .explode("doc_sections")
    .aio.extract("doc_sections")
    .explode("doc_sections_questions")
    .aio.extract("doc_sections_questions")
    .assign(
        doc_sections_type=lambda df: df.doc_sections_type.map(lambda x: x.value),
    )
)

questions_df = questions_df.assign(
    embedding=await questions_df["doc_sections_questions_question"].aio.embeddings()
)

In [8]:
from IPython.display import display, Markdown

display(Markdown(questions_df.to_markdown(index=False)))

| module                   | doc_sections_name                    | doc_sections_type   | doc_sections_description                                                                                                                                                                                                                                                                                                                                                                                                                             | doc_sections_questions_question                                                        | doc_sections_questions_answer                                                                                                                                                                                                                                                                                                                                                                                                                                                            | embedding                                                               |
|:-------------------------|:-------------------------------------|:--------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------|
| openaivec.serialize      | serialize_base_model                 | function            | Serializes a Pydantic BaseModel class into its JSON schema representation using Pydantic's built-in model_json_schema method.                                                                                                                                                                                                                                                                                                                        | What does serialize_base_model do?                                                     | It takes a Pydantic BaseModel class and returns its JSON schema as a dictionary.                                                                                                                                                                                                                                                                                                                                                                                                         | [ 0.00703712  0.02428696  0.00814858 ...  0.01711011 -0.01465855        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00564939]                                                           |
| openaivec.serialize      | serialize_base_model                 | function            | Serializes a Pydantic BaseModel class into its JSON schema representation using Pydantic's built-in model_json_schema method.                                                                                                                                                                                                                                                                                                                        | Does it serialize an instance or the class itself?                                     | It serializes the BaseModel class, not an instance.                                                                                                                                                                                                                                                                                                                                                                                                                                      | [ 0.01494039 -0.01051077 -0.00985144 ...  0.01185502 -0.02310832        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02278826]                                                           |
| openaivec.serialize      | dereference_json_schema              | function            | Recursively resolves all JSON Schema $ref references within a JSON schema dictionary by replacing them with the actual definitions from the $defs section.                                                                                                                                                                                                                                                                                           | What is the purpose of dereference_json_schema?                                        | To replace all $ref references in a JSON schema with their actual definitions, producing a fully dereferenced schema.                                                                                                                                                                                                                                                                                                                                                                    | [ 0.04457856  0.02287264  0.02154792 ...  0.02960561 -0.01883772        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00619214]                                                           |
| openaivec.serialize      | dereference_json_schema              | function            | Recursively resolves all JSON Schema $ref references within a JSON schema dictionary by replacing them with the actual definitions from the $defs section.                                                                                                                                                                                                                                                                                           | How does it handle nested references?                                                  | It recursively traverses dictionaries and lists, dereferencing any $ref it encounters.                                                                                                                                                                                                                                                                                                                                                                                                   | [-0.01308439  0.01303425  0.03325883 ... -0.00520654  0.0116234         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01113641]                                                           |
| openaivec.serialize      | dereference_json_schema              | function            | Recursively resolves all JSON Schema $ref references within a JSON schema dictionary by replacing them with the actual definitions from the $defs section.                                                                                                                                                                                                                                                                                           | What happens to the $defs key in the output?                                           | The $defs key is omitted from the returned dictionary.                                                                                                                                                                                                                                                                                                                                                                                                                                   | [0.00865954 0.01350028 0.08057138 ... 0.00208757 0.00995713 0.00438692] |
| openaivec.serialize      | parse_field                          | function            | Parses a JSON schema field definition and returns the corresponding Python type or Pydantic model type, supporting primitive types, objects, arrays, and enums.                                                                                                                                                                                                                                                                                      | How does parse_field map JSON schema types to Python types?                            | It maps 'string' to str, 'integer' to int, 'number' to float, 'boolean' to bool, 'object' to a dynamically created BaseModel, and 'array' to a List of the inner type.                                                                                                                                                                                                                                                                                                                   | [-0.00781478 -0.00236882 -0.01286039 ...  0.01962072 -0.03003714        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01659532]                                                           |
| openaivec.serialize      | parse_field                          | function            | Parses a JSON schema field definition and returns the corresponding Python type or Pydantic model type, supporting primitive types, objects, arrays, and enums.                                                                                                                                                                                                                                                                                      | How are arrays handled?                                                                | It recursively parses the 'items' field to determine the inner type and returns a List of that type.                                                                                                                                                                                                                                                                                                                                                                                     | [ 0.01308227 -0.03076579  0.00831085 ... -0.00932503 -0.01129213        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02120933]                                                           |
| openaivec.serialize      | parse_field                          | function            | Parses a JSON schema field definition and returns the corresponding Python type or Pydantic model type, supporting primitive types, objects, arrays, and enums.                                                                                                                                                                                                                                                                                      | What happens if the type is unsupported?                                               | It raises a ValueError indicating the unsupported type.                                                                                                                                                                                                                                                                                                                                                                                                                                  | [-0.0328649   0.03318971  0.04700891 ...  0.0015733  -0.03832762        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03962686]                                                           |
| openaivec.serialize      | deserialize_base_model               | function            | Dynamically creates a Pydantic BaseModel class from a JSON schema dictionary by parsing its properties, handling enums, and nested objects.                                                                                                                                                                                                                                                                                                          | How does deserialize_base_model create the model?                                      | It dereferences the JSON schema, extracts properties, and for each property creates a field with the appropriate type or enum, then uses Pydantic's create_model to generate the BaseModel subclass.                                                                                                                                                                                                                                                                                     | [-0.02071722  0.03217908  0.03344335 ...  0.02478434 -0.0173538         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00617819]                                                           |
| openaivec.serialize      | deserialize_base_model               | function            | Dynamically creates a Pydantic BaseModel class from a JSON schema dictionary by parsing its properties, handling enums, and nested objects.                                                                                                                                                                                                                                                                                                          | How are enum fields handled?                                                           | For properties with an 'enum' key, it dynamically creates a Python Enum with the given values and uses that as the field type.                                                                                                                                                                                                                                                                                                                                                           | [ 0.02811901  0.01801879  0.02084685 ... -0.04118196 -0.01253774        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.05095897]                                                           |
| openaivec.serialize      | deserialize_base_model               | function            | Dynamically creates a Pydantic BaseModel class from a JSON schema dictionary by parsing its properties, handling enums, and nested objects.                                                                                                                                                                                                                                                                                                          | How are nested objects handled?                                                        | Nested objects are parsed recursively by calling deserialize_base_model on the nested schema.                                                                                                                                                                                                                                                                                                                                                                                            | [ 0.01433957  0.00728835  0.03172003 ... -0.00496933  0.00166255        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01938911]                                                           |
| openaivec.responses      | _vectorize_system_message            | function            | Generates a composite system prompt that instructs the language model to handle multiple user messages in a single batch. It embeds the original system message and adds XML-like instructions explaining how the model should process an array of JSON user messages and produce corresponding outputs in order.                                                                                                                                    | What is the purpose of the _vectorize_system_message function?                         | It creates a special system prompt that teaches the language model how to process multiple user messages sent as a batch in JSON format and how to respond with a corresponding batch of outputs.                                                                                                                                                                                                                                                                                        | [ 0.01709459  0.05447612 -0.0606034  ...  0.01939875 -0.02363378        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02018397]                                                           |
| openaivec.responses      | _vectorize_system_message            | function            | Generates a composite system prompt that instructs the language model to handle multiple user messages in a single batch. It embeds the original system message and adds XML-like instructions explaining how the model should process an array of JSON user messages and produce corresponding outputs in order.                                                                                                                                    | What does the returned prompt contain?                                                 | It contains the original system message wrapped inside XML-like tags, additional instructions for batch processing, and an example input-output format demonstrating how multiple messages should be handled.                                                                                                                                                                                                                                                                            | [ 0.02789079  0.01671465  0.02525611 ... -0.02232397  0.01041124        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.02198401]                                                           |
| openaivec.responses      | Message                              | class               | A generic Pydantic model representing a message with an integer ID and a body of generic type T. Used to encapsulate user or assistant messages with stable identifiers.                                                                                                                                                                                                                                                                             | What is the purpose of the Message class?                                              | It models a message with an ID and a body, where the body can be of any type, allowing consistent handling of messages in batches.                                                                                                                                                                                                                                                                                                                                                       | [ 0.0006052   0.02340852 -0.0089033  ...  0.01004708 -0.02307058        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00352557]                                                           |
| openaivec.responses      | Message                              | class               | A generic Pydantic model representing a message with an integer ID and a body of generic type T. Used to encapsulate user or assistant messages with stable identifiers.                                                                                                                                                                                                                                                                             | Why is it generic?                                                                     | To allow the body to be typed flexibly, for example, user messages have string bodies, while assistant messages can have bodies of different types.                                                                                                                                                                                                                                                                                                                                      | [-0.01150875 -0.01637515  0.00045024 ...  0.00285542 -0.02634302        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.05317355]                                                           |
| openaivec.responses      | Request                              | class               | A Pydantic model representing a request containing a list of user messages, each wrapped in a Message with a string body.                                                                                                                                                                                                                                                                                                                            | What does the Request class represent?                                                 | It represents the input structure sent to the language model, containing multiple user messages each with an ID and string content.                                                                                                                                                                                                                                                                                                                                                      | [-0.0218132   0.01502862 -0.01244496 ...  0.01421342  0.0092236         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00130662]                                                           |
| openaivec.responses      | Response                             | class               | A generic Pydantic model representing the response from the language model, containing a list of assistant messages each wrapped in a Message with a body of generic type T.                                                                                                                                                                                                                                                                         | What is the purpose of the Response class?                                             | It models the output structure from the language model, holding multiple assistant messages with their IDs and typed bodies.                                                                                                                                                                                                                                                                                                                                                             | [-0.04213437  0.04391241  0.00076653 ...  0.03294778 -0.01270227        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.02025898]                                                           |
| openaivec.responses      | VectorizedResponses                  | class               | An abstract base class defining the interface for batched language model interactions. It requires implementing a parse method that takes a list of input strings and a batch size, returning a list of outputs in the original order.                                                                                                                                                                                                               | What is the role of the VectorizedResponses class?                                     | It defines a minimal interface for classes that provide batched LLM responses, ensuring they implement a parse method.                                                                                                                                                                                                                                                                                                                                                                   | [-0.00268146  0.05417196 -0.0008065  ...  0.03685673 -0.02015989        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01008681]                                                           |
| openaivec.responses      | VectorizedResponses                  | class               | An abstract base class defining the interface for batched language model interactions. It requires implementing a parse method that takes a list of input strings and a batch size, returning a list of outputs in the original order.                                                                                                                                                                                                               | What does the parse method do?                                                         | It accepts multiple user prompt strings and a batch size, and returns model outputs corresponding to each input in the original order.                                                                                                                                                                                                                                                                                                                                                   | [-0.02220551  0.00792778 -0.00546444 ...  0.00282238 -0.02663631        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00561578]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI            | class               | A stateless wrapper around the OpenAI JSON-mode API that enables sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic handling of rate limits with retries.                                                                                                                                                                                           | What is the purpose of VectorizedResponsesOpenAI?                                      | It provides a convenient interface to send multiple prompts to OpenAI's JSON-mode API in batches and get responses in the same order, improving efficiency and reducing API calls.                                                                                                                                                                                                                                                                                                       | [-0.01109683  0.051934    0.02201785 ...  0.03973491 -0.01994861        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.0090614 ]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI            | class               | A stateless wrapper around the OpenAI JSON-mode API that enables sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic handling of rate limits with retries.                                                                                                                                                                                           | What are the key attributes of VectorizedResponsesOpenAI?                              | Attributes include the OpenAI client, model name, system message, temperature, top_p, response format type, and a flag for parallel execution.                                                                                                                                                                                                                                                                                                                                           | [-0.01390528  0.05879487  0.01632593 ...  0.03095741 -0.00488837        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01547871]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI            | class               | A stateless wrapper around the OpenAI JSON-mode API that enables sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic handling of rate limits with retries.                                                                                                                                                                                           | How does VectorizedResponsesOpenAI handle system messages?                             | It uses the _vectorize_system_message function to embed the original system message into a composite prompt that instructs the model on batch processing.                                                                                                                                                                                                                                                                                                                                | [-0.00989294  0.04338094  0.03008988 ...  0.03628324 -0.00290143        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01405838]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI            | class               | A stateless wrapper around the OpenAI JSON-mode API that enables sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic handling of rate limits with retries.                                                                                                                                                                                           | What does the _request_llm method do?                                                  | It makes a single API call to the OpenAI JSON-mode endpoint with a batch of user messages, parses the response into a typed Pydantic model, and handles rate limit errors with exponential backoff.                                                                                                                                                                                                                                                                                      | [-0.01770798 -0.00397584  0.02455996 ... -0.0109406   0.00352644        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03251163]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI            | class               | A stateless wrapper around the OpenAI JSON-mode API that enables sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic handling of rate limits with retries.                                                                                                                                                                                           | How does _predict_chunk work?                                                          | It converts a list of user prompt strings into Message objects with stable IDs, calls _request_llm to get responses, and then reorders the responses to match the original input order.                                                                                                                                                                                                                                                                                                  | [ 0.00891291 -0.02294876 -0.02211336 ... -0.02371044 -0.01006772        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00382684]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI            | class               | A stateless wrapper around the OpenAI JSON-mode API that enables sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic handling of rate limits with retries.                                                                                                                                                                                           | What is the role of the parse method?                                                  | It is the public API that accepts all input prompts, optionally executes minibatches in parallel or serially, and returns the assistant responses in the same order as inputs, handling deduplication internally.                                                                                                                                                                                                                                                                        | [-0.00645258  0.02360761 -0.026109   ... -0.00639658 -0.03594032        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00126547]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI            | class               | A stateless wrapper around the OpenAI JSON-mode API that enables sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic handling of rate limits with retries.                                                                                                                                                                                           | How does the class handle duplicate inputs?                                            | Duplicates are de-duplicated internally to avoid redundant API calls, but the final output preserves the original input order and duplicates.                                                                                                                                                                                                                                                                                                                                            | [ 0.04447343  0.00583695  0.05890285 ...  0.01148132  0.00773322        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00539251]                                                           |
| openaivec.responses      | VectorizedResponsesOpenAI            | class               | A stateless wrapper around the OpenAI JSON-mode API that enables sending multiple user prompts in a single request and receiving ordered responses. It supports batching, optional parallel execution, and automatic handling of rate limits with retries.                                                                                                                                                                                           | What is the significance of the is_parallel flag?                                      | If set to True, minibatches are processed concurrently to improve throughput; otherwise, they are processed serially.                                                                                                                                                                                                                                                                                                                                                                    | [ 0.01089032 -0.00940681 -0.00049099 ...  0.00516195 -0.02600191        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.043858  ]                                                           |
| openaivec.log            | observe                              | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                                                                             | What is the purpose of the observe function?                                           | The observe function is a decorator factory that returns a decorator to wrap class methods, enabling logging of method invocation start and end events with detailed contextual information.                                                                                                                                                                                                                                                                                             | [ 0.00277905  0.03274433 -0.04986133 ... -0.01011139 -0.02127679        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02102383]                                                           |
| openaivec.log            | observe                              | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                                                                             | How does the decorator log the start and end of a method call?                         | It logs a JSON-formatted message before the method execution indicating the start, including a unique transaction ID, class name, method name, and timestamp. After the method completes (even if an exception occurs), it logs a similar message indicating the end of the method call.                                                                                                                                                                                                 | [ 0.03208767  0.02015764  0.02172602 ... -0.03283329 -0.01457829        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.02116038]                                                           |
| openaivec.log            | observe                              | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                                                                             | What is the role of the transaction_id in the logs?                                    | The transaction_id is a unique UUID string generated for each method call to correlate the start and end log entries for the same invocation.                                                                                                                                                                                                                                                                                                                                            | [-0.01762005  0.0067519  -0.01328329 ... -0.00216969  0.01564593        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00634238]                                                           |
| openaivec.log            | observe                              | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                                                                             | How are the logger instances structured?                                               | The decorator creates a child logger from the provided logger using the class name, then further creates a child logger from that using the method name, allowing hierarchical and contextual logging.                                                                                                                                                                                                                                                                                   | [-0.00484941  0.00160006  0.03589756 ... -0.01211121 -0.0008475         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0081937 ]                                                           |
| openaivec.log            | observe                              | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                                                                             | Why is functools.wraps used in the decorator?                                          | functools.wraps preserves the metadata of the original function (such as its name and docstring) when it is wrapped by the decorator, which is important for introspection and debugging.                                                                                                                                                                                                                                                                                                | [ 0.02342422 -0.00924928 -0.02813095 ... -0.0250661  -0.00478336        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00316336]                                                           |
| openaivec.log            | observe                              | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                                                                             | What arguments does the decorated function accept?                                     | The decorated function accepts self (the instance of the class), positional arguments (*args), and keyword arguments (**kwargs), forwarding them to the original method.                                                                                                                                                                                                                                                                                                                 | [ 0.04615695  0.01306537  0.00796905 ... -0.01817551  0.01385952        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00582486]                                                           |
| openaivec.log            | observe                              | function            | A decorator factory that creates a decorator to log the start and end of method calls within a class. It uses a provided Logger instance to create child loggers specific to the class and method names, logs a unique transaction ID, timestamps, and the event type ('start' or 'end') in JSON format.                                                                                                                                             | What happens if the decorated method raises an exception?                              | The 'end' log entry is still recorded in the finally block, ensuring that the end of the method call is logged regardless of whether the method completes successfully or raises an exception.                                                                                                                                                                                                                                                                                           | [ 0.01342502  0.00332435  0.05860049 ... -0.00578412 -0.01651329        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00790571]                                                           |
| openaivec.util           | split_to_minibatch                   | function            | Splits a list into smaller sublists (mini-batches) each with a maximum specified size. Useful for processing large lists in manageable chunks.                                                                                                                                                                                                                                                                                                       | What does split_to_minibatch do?                                                       | It divides a list into consecutive sublists, each with up to batch_size elements, except possibly the last which may be smaller.                                                                                                                                                                                                                                                                                                                                                         | [-0.03428983 -0.02654952  0.03640322 ... -0.00976785 -0.00518112        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01397482]                                                           |
| openaivec.util           | split_to_minibatch                   | function            | Splits a list into smaller sublists (mini-batches) each with a maximum specified size. Useful for processing large lists in manageable chunks.                                                                                                                                                                                                                                                                                                       | What is the return type of split_to_minibatch?                                         | It returns a list of lists, where each inner list is a mini-batch of the original list elements.                                                                                                                                                                                                                                                                                                                                                                                         | [-0.0423137   0.01366646  0.02438525 ... -0.01129939  0.01834955        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00097937]                                                           |
| openaivec.util           | map_minibatch                        | function            | Applies a given function to each mini-batch of a list sequentially and concatenates the results into a single flattened list.                                                                                                                                                                                                                                                                                                                        | How does map_minibatch process the input list?                                         | It first splits the list into mini-batches, applies the function f to each batch sequentially, and then flattens the results into one list.                                                                                                                                                                                                                                                                                                                                              | [-5.08639254e-02 -7.87892193e-03  5.18612564e-02 ... -7.13092275e-03    |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   8.51627905e-04 -1.39519325e-05]                                       |
| openaivec.util           | map_minibatch                        | function            | Applies a given function to each mini-batch of a list sequentially and concatenates the results into a single flattened list.                                                                                                                                                                                                                                                                                                                        | What is the purpose of map_minibatch?                                                  | To process large lists in batches to manage memory or API limits, while collecting all results in order.                                                                                                                                                                                                                                                                                                                                                                                 | [-0.03826397 -0.01033744  0.04106691 ... -0.01198964 -0.00972028        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0039569 ]                                                           |
| openaivec.util           | map_minibatch_async                  | function            | Asynchronous version of map_minibatch that concurrently applies an async function to each mini-batch and gathers the results.                                                                                                                                                                                                                                                                                                                        | How does map_minibatch_async differ from map_minibatch?                                | It uses asyncio to run the function f concurrently on all mini-batches, improving efficiency for IO-bound async operations.                                                                                                                                                                                                                                                                                                                                                              | [-0.04970649 -0.02841983  0.03476294 ...  0.00099958 -0.01059254        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00847629]                                                           |
| openaivec.util           | map_minibatch_async                  | function            | Asynchronous version of map_minibatch that concurrently applies an async function to each mini-batch and gathers the results.                                                                                                                                                                                                                                                                                                                        | What does map_minibatch_async return?                                                  | It returns a flattened list of results after asynchronously processing all batches.                                                                                                                                                                                                                                                                                                                                                                                                      | [-0.05538616 -0.01444485  0.03572749 ... -0.00601055 -0.00694769        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01394422]                                                           |
| openaivec.util           | map_minibatch_parallel               | function            | Parallel variant of map_minibatch that uses a thread pool to apply a function to each mini-batch concurrently in separate threads.                                                                                                                                                                                                                                                                                                                   | How does map_minibatch_parallel achieve parallelism?                                   | It uses ThreadPoolExecutor to run the function f on each batch in parallel threads.                                                                                                                                                                                                                                                                                                                                                                                                      | [-0.04252512 -0.01124508  0.05968773 ... -0.00344742 -0.01473124        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00368877]                                                           |
| openaivec.util           | map_minibatch_parallel               | function            | Parallel variant of map_minibatch that uses a thread pool to apply a function to each mini-batch concurrently in separate threads.                                                                                                                                                                                                                                                                                                                   | When should map_minibatch_parallel be used?                                            | When the function f is CPU-bound or blocking and can benefit from parallel execution in threads.                                                                                                                                                                                                                                                                                                                                                                                         | [-0.0349329  -0.01356424  0.06353249 ... -0.00701899  0.00132697        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0144494 ]                                                           |
| openaivec.util           | map_unique                           | function            | Applies a function once per unique value in a list and broadcasts the results back to align with the original list order, avoiding redundant computations.                                                                                                                                                                                                                                                                                           | What problem does map_unique solve?                                                    | It avoids repeated computation for duplicate values by processing unique values only and mapping results back.                                                                                                                                                                                                                                                                                                                                                                           | [ 0.010185   -0.01097008  0.04267466 ...  0.00783677  0.00210289        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00703767]                                                           |
| openaivec.util           | map_unique                           | function            | Applies a function once per unique value in a list and broadcasts the results back to align with the original list order, avoiding redundant computations.                                                                                                                                                                                                                                                                                           | How does map_unique maintain the original order?                                       | It uses a dictionary to map each unique value to its result index and reconstructs the output list accordingly.                                                                                                                                                                                                                                                                                                                                                                          | [ 0.01759752 -0.0185014   0.02182036 ... -0.01151748  0.00749236        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01128444]                                                           |
| openaivec.util           | map_unique_minibatch                 | function            | Combines map_unique and map_minibatch to apply a function to unique values in mini-batches, then broadcasts results back to the original list order.                                                                                                                                                                                                                                                                                                 | How does map_unique_minibatch process the input list?                                  | It extracts unique values, processes them in mini-batches with the function f, then maps results back to the original list.                                                                                                                                                                                                                                                                                                                                                              | [-0.04431516 -0.02881766  0.04042158 ... -0.01072017  0.00480935        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0031059 ]                                                           |
| openaivec.util           | map_unique_minibatch                 | function            | Combines map_unique and map_minibatch to apply a function to unique values in mini-batches, then broadcasts results back to the original list order.                                                                                                                                                                                                                                                                                                 | Why combine uniqueness and mini-batching?                                              | To reduce redundant computation on duplicates and handle large unique sets efficiently in batches.                                                                                                                                                                                                                                                                                                                                                                                       | [ 0.01518826 -0.05751904  0.06225455 ... -0.02211827 -0.02402402        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00604571]                                                           |
| openaivec.util           | map_unique_minibatch_async           | function            | Asynchronous version of map_unique_minibatch that concurrently applies an async function to mini-batches of unique values and broadcasts results back.                                                                                                                                                                                                                                                                                               | What is the advantage of map_unique_minibatch_async?                                   | It efficiently processes unique values asynchronously in batches, improving performance for async operations.                                                                                                                                                                                                                                                                                                                                                                            | [-0.0221184  -0.04338077  0.04370808 ... -0.01442669 -0.01041088        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00085053]                                                           |
| openaivec.util           | map_unique_minibatch_async           | function            | Asynchronous version of map_unique_minibatch that concurrently applies an async function to mini-batches of unique values and broadcasts results back.                                                                                                                                                                                                                                                                                               | How does it ensure results align with the original list?                               | By mapping unique values to their indices and reconstructing the output list accordingly after async batch processing.                                                                                                                                                                                                                                                                                                                                                                   | [-0.01255142  0.01424044  0.06935821 ... -0.00651976 -0.03157999        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.0175255 ]                                                           |
| openaivec.util           | map_unique_minibatch_parallel        | function            | Parallel version of map_unique_minibatch that uses threads to process mini-batches of unique values concurrently and broadcasts results back.                                                                                                                                                                                                                                                                                                        | How does map_unique_minibatch_parallel achieve concurrency?                            | It uses ThreadPoolExecutor to run the function f on batches of unique values in parallel threads.                                                                                                                                                                                                                                                                                                                                                                                        | [-0.02963243 -0.0401325   0.05679128 ... -0.02160593 -0.01121312        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00013389]                                                           |
| openaivec.util           | map_unique_minibatch_parallel        | function            | Parallel version of map_unique_minibatch that uses threads to process mini-batches of unique values concurrently and broadcasts results back.                                                                                                                                                                                                                                                                                                        | When is this function useful?                                                          | When processing unique values with a CPU-bound or blocking function that benefits from parallel execution.                                                                                                                                                                                                                                                                                                                                                                               | [ 0.0011816  -0.00329537  0.01710281 ...  0.01617833 -0.0242675         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.04264916]                                                           |
| openaivec.util           | get_exponential_with_cutoff          | function            | Samples a random value from an exponential distribution with a specified scale, rejecting samples above three times the scale to impose an upper cutoff.                                                                                                                                                                                                                                                                                             | What distribution does get_exponential_with_cutoff sample from?                        | An exponential distribution with rate 1/scale, truncated at 3 * scale.                                                                                                                                                                                                                                                                                                                                                                                                                   | [ 0.02913438 -0.00782106  0.03667126 ... -0.01777963 -0.00064944        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0120343 ]                                                           |
| openaivec.util           | get_exponential_with_cutoff          | function            | Samples a random value from an exponential distribution with a specified scale, rejecting samples above three times the scale to impose an upper cutoff.                                                                                                                                                                                                                                                                                             | Why impose a cutoff at 3 * scale?                                                      | To limit the maximum sampled value, preventing excessively large backoff intervals.                                                                                                                                                                                                                                                                                                                                                                                                      | [ 0.03122566  0.01044959  0.08008789 ...  0.0198219  -0.01243486        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02379244]                                                           |
| openaivec.util           | backoff                              | function            | Decorator that adds exponential back-off retry logic to a function, retrying on a specified exception with randomized delays and optional maximum retries.                                                                                                                                                                                                                                                                                           | What does the backoff decorator do?                                                    | It retries the decorated function when a specified exception occurs, waiting for exponentially distributed intervals between attempts.                                                                                                                                                                                                                                                                                                                                                   | [ 0.06094427 -0.00867174 -0.0119895  ... -0.01720109 -0.0267272         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.0132568 ]                                                           |
| openaivec.util           | backoff                              | function            | Decorator that adds exponential back-off retry logic to a function, retrying on a specified exception with randomized delays and optional maximum retries.                                                                                                                                                                                                                                                                                           | How are retry intervals determined?                                                    | Using get_exponential_with_cutoff to sample delays with an upper bound, optionally scaled by a parameter.                                                                                                                                                                                                                                                                                                                                                                                | [-0.05430131  0.01052021  0.05299956 ... -0.03172004 -0.00899266        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02165145]                                                           |
| openaivec.util           | backoff                              | function            | Decorator that adds exponential back-off retry logic to a function, retrying on a specified exception with randomized delays and optional maximum retries.                                                                                                                                                                                                                                                                                           | What happens when max_retries is exceeded?                                             | The exception is re-raised and no further retries occur.                                                                                                                                                                                                                                                                                                                                                                                                                                 | [ 0.00192901  0.05209647  0.1006276  ... -0.02105943  0.01135453        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03384408]                                                           |
| openaivec.util           | TextChunker                          | class               | Utility class for splitting text into chunks bounded by a maximum token count, using token-aware sentence segmentation based on specified separators.                                                                                                                                                                                                                                                                                                | What is the purpose of the TextChunker class?                                          | To split text into chunks that do not exceed a specified token limit, respecting sentence boundaries.                                                                                                                                                                                                                                                                                                                                                                                    | [-0.00927683  0.01474658  0.00526999 ... -0.01153289 -0.03569731        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01106875]                                                           |
| openaivec.util           | TextChunker                          | class               | Utility class for splitting text into chunks bounded by a maximum token count, using token-aware sentence segmentation based on specified separators.                                                                                                                                                                                                                                                                                                | How does the split method work?                                                        | It splits text by given separators, encodes each segment to count tokens, and greedily packs segments into chunks without exceeding max_tokens.                                                                                                                                                                                                                                                                                                                                          | [-0.0067128   0.01952397  0.0252383  ... -0.00917058 -0.03370227        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01186646]                                                           |
| openaivec.util           | TextChunker                          | class               | Utility class for splitting text into chunks bounded by a maximum token count, using token-aware sentence segmentation based on specified separators.                                                                                                                                                                                                                                                                                                | What is the role of the enc attribute?                                                 | It is a tiktoken.Encoding instance used to encode text segments and count tokens accurately.                                                                                                                                                                                                                                                                                                                                                                                             | [ 0.03426085  0.04239821  0.04572419 ... -0.02218398 -0.01955226        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01738876]                                                           |
| openaivec.embeddings     | VectorizedEmbeddings                 | class               | An abstract base class defining the interface for a sentence-embedding backend. It declares an abstract method `create` which must be implemented by subclasses to embed a list of input sentences into vectors.                                                                                                                                                                                                                                     | What is the purpose of the VectorizedEmbeddings class?                                 | It serves as an abstract interface that defines the contract for any sentence embedding backend, ensuring that subclasses implement the `create` method to embed sentences.                                                                                                                                                                                                                                                                                                              | [-0.00587412  0.03136895 -0.01575703 ...  0.0246453  -0.0402935         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00839851]                                                           |
| openaivec.embeddings     | VectorizedEmbeddings                 | class               | An abstract base class defining the interface for a sentence-embedding backend. It declares an abstract method `create` which must be implemented by subclasses to embed a list of input sentences into vectors.                                                                                                                                                                                                                                     | What does the create method do?                                                        | The `create` method takes a list of input strings and a batch size, and returns a list of numpy float32 arrays representing the embeddings of each input sentence. It is abstract and must be implemented by subclasses.                                                                                                                                                                                                                                                                 | [ 0.01736155  0.01605976  0.01950113 ... -0.03392398 -0.02510787        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00256653]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI           | class               | A concrete implementation of VectorizedEmbeddings that uses the OpenAI embeddings endpoint to generate sentence embeddings. It supports both sequential and parallel execution and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                                                                                                               | What attributes does VectorizedEmbeddingsOpenAI have?                                  | It has three attributes: `client` (an OpenAI client instance), `model_name` (the identifier of the embedding model to use), and `is_parallel` (a boolean indicating whether to run embedding requests in parallel).                                                                                                                                                                                                                                                                      | [-0.00371451  0.02944302  0.01528008 ...  0.02053067 -0.03051051        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00261754]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI           | class               | A concrete implementation of VectorizedEmbeddings that uses the OpenAI embeddings endpoint to generate sentence embeddings. It supports both sequential and parallel execution and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                                                                                                               | How does the _embed_chunk method work?                                                 | The `_embed_chunk` method sends a batch of input sentences to the OpenAI embeddings endpoint using the configured client and model. It returns a list of numpy float32 arrays representing the embeddings. It is decorated to automatically retry with exponential backoff if a RateLimitError occurs, and it logs its activity.                                                                                                                                                         | [-0.01645049  0.00669724 -0.0173738  ... -0.0383368  -0.02513739        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01101468]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI           | class               | A concrete implementation of VectorizedEmbeddings that uses the OpenAI embeddings endpoint to generate sentence embeddings. It supports both sequential and parallel execution and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                                                                                                               | What is the purpose of the create method in VectorizedEmbeddingsOpenAI?                | The `create` method implements the abstract method from the base class. It processes the input sentences in batches of the specified size, either sequentially or in parallel depending on the `is_parallel` flag. It uses helper functions `map_unique_minibatch` or `map_unique_minibatch_parallel` to handle batching and deduplication, delegating the actual embedding work to `_embed_chunk`. It also logs its activity and propagates rate limit errors if retries are exhausted. | [ 0.029313    0.02754226  0.0047858  ...  0.01434542 -0.03249555        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00574595]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI           | class               | A concrete implementation of VectorizedEmbeddings that uses the OpenAI embeddings endpoint to generate sentence embeddings. It supports both sequential and parallel execution and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                                                                                                               | How does the class handle OpenAI rate limits?                                          | The `_embed_chunk` method is decorated with a `backoff` decorator that catches `RateLimitError` exceptions and retries the request with exponential backoff, scaling the wait time by 60 seconds and allowing up to 16 retries.                                                                                                                                                                                                                                                          | [-0.00045758  0.01935145  0.08180828 ...  0.00658284 -0.00877711        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00104054]                                                           |
| openaivec.embeddings     | VectorizedEmbeddingsOpenAI           | class               | A concrete implementation of VectorizedEmbeddings that uses the OpenAI embeddings endpoint to generate sentence embeddings. It supports both sequential and parallel execution and applies an exponential backoff strategy to handle rate limiting errors from OpenAI.                                                                                                                                                                               | What is the role of the observe decorator?                                             | The `observe` decorator is used to log or monitor the execution of the decorated methods (`_embed_chunk` and `create`), using the module's logger to track their activity and possibly performance or errors.                                                                                                                                                                                                                                                                            | [ 0.0259248   0.01859454 -0.04992506 ... -0.03413244 -0.02295307        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0069305 ]                                                           |
| openaivec.pandas_ext     | use                                  | function            | Registers a custom OpenAI-compatible client instance to be used by the pandas extension helpers. This client instance is stored globally and reused for all subsequent calls.                                                                                                                                                                                                                                                                        | What is the purpose of the 'use' function?                                             | The 'use' function allows the user to register a pre-configured OpenAI or AzureOpenAI client instance, which will be reused by all helper functions in the module to interact with the OpenAI API.                                                                                                                                                                                                                                                                                       | [-0.00614469  0.03181943 -0.02008159 ...  0.03139517 -0.06346916        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00277713]                                                           |
| openaivec.pandas_ext     | use                                  | function            | Registers a custom OpenAI-compatible client instance to be used by the pandas extension helpers. This client instance is stored globally and reused for all subsequent calls.                                                                                                                                                                                                                                                                        | What type of client can be passed to 'use'?                                            | The client must be an instance of 'openai.OpenAI' or 'openai.AzureOpenAI', which are compatible with the OpenAI API.                                                                                                                                                                                                                                                                                                                                                                     | [-0.02636235  0.02433772  0.02907591 ... -0.01038325 -0.05258409        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01781392]                                                           |
| openaivec.pandas_ext     | use_openai                           | function            | Creates and registers a default OpenAI client using a provided API key. This simplifies setup by internally creating the OpenAI client instance and storing it globally for reuse.                                                                                                                                                                                                                                                                   | What does 'use_openai' do?                                                             | It creates an OpenAI client with the given API key and registers it globally for use by the pandas extension.                                                                                                                                                                                                                                                                                                                                                                            | [-0.00307013  0.00817355  0.03673382 ...  0.02138318 -0.04290102        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00997119]                                                           |
| openaivec.pandas_ext     | use_openai                           | function            | Creates and registers a default OpenAI client using a provided API key. This simplifies setup by internally creating the OpenAI client instance and storing it globally for reuse.                                                                                                                                                                                                                                                                   | What argument does 'use_openai' require?                                               | It requires an 'api_key' string which is used to authenticate with the OpenAI API.                                                                                                                                                                                                                                                                                                                                                                                                       | [ 0.0173002   0.02955223  0.01536279 ...  0.02709636 -0.01358911        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00560073]                                                           |
| openaivec.pandas_ext     | use_azure_openai                     | function            | Creates and registers an Azure OpenAI client using the provided Azure subscription key, endpoint, and API version. This allows integration with Azure-hosted OpenAI services.                                                                                                                                                                                                                                                                        | What parameters are needed for 'use_azure_openai'?                                     | It requires 'api_key' (Azure subscription key), 'endpoint' (resource endpoint URL), and 'api_version' (REST API version string).                                                                                                                                                                                                                                                                                                                                                         | [-0.00893216 -0.00316293  0.05608566 ... -0.01587795  0.02262901        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00157579]                                                           |
| openaivec.pandas_ext     | use_azure_openai                     | function            | Creates and registers an Azure OpenAI client using the provided Azure subscription key, endpoint, and API version. This allows integration with Azure-hosted OpenAI services.                                                                                                                                                                                                                                                                        | What does this function configure?                                                     | It configures and registers an AzureOpenAI client globally for use by the pandas extension.                                                                                                                                                                                                                                                                                                                                                                                              | [-0.00182141  0.01800103  0.04221722 ... -0.00731068 -0.0272092         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00887879]                                                           |
| openaivec.pandas_ext     | responses_model                      | function            | Overrides the model used for generating text responses from the OpenAI API. It also updates the token encoding used for token counting based on the new model name.                                                                                                                                                                                                                                                                                  | What is the default model for responses?                                               | The default model is 'gpt-4o-mini'.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | [-0.03844502  0.03547044  0.0389782  ... -0.00158463  0.00072084        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0087273 ]                                                           |
| openaivec.pandas_ext     | responses_model                      | function            | Overrides the model used for generating text responses from the OpenAI API. It also updates the token encoding used for token counting based on the new model name.                                                                                                                                                                                                                                                                                  | What happens if the model name is not supported by tiktoken?                           | A warning is logged and the encoding falls back to 'o200k_base'.                                                                                                                                                                                                                                                                                                                                                                                                                         | [-0.00124235  0.00084584  0.00917826 ...  0.02310167  0.00044242        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03411298]                                                           |
| openaivec.pandas_ext     | embeddings_model                     | function            | Overrides the model used for generating text embeddings from the OpenAI API.                                                                                                                                                                                                                                                                                                                                                                         | What is the default embeddings model?                                                  | The default embeddings model is 'text-embedding-3-small'.                                                                                                                                                                                                                                                                                                                                                                                                                                | [-0.01520886 -0.01832679  0.04243724 ... -0.00143699 -0.01405451        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00303463]                                                           |
| openaivec.pandas_ext     | embeddings_model                     | function            | Overrides the model used for generating text embeddings from the OpenAI API.                                                                                                                                                                                                                                                                                                                                                                         | How do you change the embeddings model?                                                | By calling 'embeddings_model' with the desired model name string.                                                                                                                                                                                                                                                                                                                                                                                                                        | [-0.00491218 -0.01635941  0.0366062  ... -0.01440326 -0.01307008        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00121403]                                                           |
| openaivec.pandas_ext     | _get_openai_client                   | function            | Retrieves the globally registered OpenAI client. If none is registered, attempts to create one from environment variables for either OpenAI or Azure OpenAI. Raises an error if no credentials are found.                                                                                                                                                                                                                                            | What environment variables does this function check?                                   | It checks 'OPENAI_API_KEY' for OpenAI, and 'AZURE_OPENAI_API_KEY', 'AZURE_OPENAI_ENDPOINT', and 'AZURE_OPENAI_API_VERSION' for Azure OpenAI.                                                                                                                                                                                                                                                                                                                                             | [-0.03151957  0.02745681  0.05549781 ... -0.04564629 -0.01579962        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00952624]                                                           |
| openaivec.pandas_ext     | _get_openai_client                   | function            | Retrieves the globally registered OpenAI client. If none is registered, attempts to create one from environment variables for either OpenAI or Azure OpenAI. Raises an error if no credentials are found.                                                                                                                                                                                                                                            | What happens if no client is registered and no environment variables are set?          | It raises a ValueError indicating that no OpenAI API key was found and instructs the user to set the appropriate environment variables.                                                                                                                                                                                                                                                                                                                                                  | [-0.04091008  0.03367254  0.09326676 ... -0.04031978  0.0024478         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01569417]                                                           |
| openaivec.pandas_ext     | _extract_value                       | function            | Converts a single Series element into a homogeneous dictionary representation. Supports Pydantic BaseModel instances and dicts. Returns an empty dict if the value cannot be coerced.                                                                                                                                                                                                                                                                | What types of values can '_extract_value' handle?                                      | It can handle None, Pydantic BaseModel instances, and dicts.                                                                                                                                                                                                                                                                                                                                                                                                                             | [-0.00045624  0.04830818  0.0065586  ...  0.01156631  0.01763909        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02025506]                                                           |
| openaivec.pandas_ext     | _extract_value                       | function            | Converts a single Series element into a homogeneous dictionary representation. Supports Pydantic BaseModel instances and dicts. Returns an empty dict if the value cannot be coerced.                                                                                                                                                                                                                                                                | What does it return if the value is not a dict or BaseModel?                           | It logs a warning and returns an empty dictionary.                                                                                                                                                                                                                                                                                                                                                                                                                                       | [-0.03401181  0.04106661  0.01299391 ...  0.01865127 -0.00584895        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.04090884]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor              | class               | A pandas Series accessor registered as '.ai' that adds OpenAI-related helper methods to pandas Series objects. It enables calling OpenAI models for responses, embeddings, token counting, and extracting structured data from Series elements.                                                                                                                                                                                                      | How do you access the OpenAI helpers on a pandas Series?                               | By using the '.ai' accessor, e.g., 'series.ai.responses(...)'.                                                                                                                                                                                                                                                                                                                                                                                                                           | [-0.01635489 -0.03776291  0.07863165 ...  0.01122781 -0.01568936        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00983512]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor              | class               | A pandas Series accessor registered as '.ai' that adds OpenAI-related helper methods to pandas Series objects. It enables calling OpenAI models for responses, embeddings, token counting, and extracting structured data from Series elements.                                                                                                                                                                                                      | What methods does this accessor provide?                                               | It provides 'responses' to get LLM responses per element, 'embeddings' to get embeddings, 'count_tokens' to count tokens per element, and 'extract' to expand dict or BaseModel elements into DataFrame columns.                                                                                                                                                                                                                                                                         | [ 0.00432435  0.01770993  0.04900433 ... -0.0284508  -0.0166049         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0297179 ]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.responses    | function            | Generates a text response from an LLM for each element in the Series, using a system prompt and optional response format. Supports batching for efficiency.                                                                                                                                                                                                                                                                                          | What does the 'instructions' parameter do?                                             | It provides a system prompt prepended to every user message sent to the LLM.                                                                                                                                                                                                                                                                                                                                                                                                             | [-1.5896667e-02  1.3629593e-02  3.7576407e-02 ... -7.2649025e-05        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -2.5657301e-03 -3.5363636e-03]                                         |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.responses    | function            | Generates a text response from an LLM for each element in the Series, using a system prompt and optional response format. Supports batching for efficiency.                                                                                                                                                                                                                                                                                          | What is the default response format?                                                   | The default response format is 'str', returning plain text responses.                                                                                                                                                                                                                                                                                                                                                                                                                    | [-0.06607127 -0.01000901  0.03518557 ... -0.00139369 -0.00021303        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01595411]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.responses    | function            | Generates a text response from an LLM for each element in the Series, using a system prompt and optional response format. Supports batching for efficiency.                                                                                                                                                                                                                                                                                          | How does batching work in this method?                                                 | Elements are grouped into batches of size 'batch_size' and sent in parallel requests to the OpenAI API.                                                                                                                                                                                                                                                                                                                                                                                  | [-0.02998115 -0.05117995  0.02429602 ... -0.01823922 -0.0219146         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00416405]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.embeddings   | function            | Computes OpenAI embeddings for each element in the Series, returning a Series of numpy arrays representing the embedding vectors.                                                                                                                                                                                                                                                                                                                    | What is the default embedding model used?                                              | The default embedding model is 'text-embedding-3-small'.                                                                                                                                                                                                                                                                                                                                                                                                                                 | [-0.01314448 -0.00724992  0.05610697 ... -0.01502409 -0.01179551        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00221525]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.embeddings   | function            | Computes OpenAI embeddings for each element in the Series, returning a Series of numpy arrays representing the embedding vectors.                                                                                                                                                                                                                                                                                                                    | What is the return type of this method?                                                | It returns a pandas Series where each value is a numpy ndarray of dtype float32 representing the embedding vector.                                                                                                                                                                                                                                                                                                                                                                       | [-0.01613531  0.05200347 -0.0045003  ...  0.00926236  0.00843009        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00892677]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.count_tokens | function            | Counts the number of tokens in each Series element using the tiktoken library and the current response model's encoding.                                                                                                                                                                                                                                                                                                                             | What library is used for token counting?                                               | The 'tiktoken' library is used.                                                                                                                                                                                                                                                                                                                                                                                                                                                          | [-0.02452642 -0.00406747  0.03583695 ... -0.01786442 -0.00565189        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02391833]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.count_tokens | function            | Counts the number of tokens in each Series element using the tiktoken library and the current response model's encoding.                                                                                                                                                                                                                                                                                                                             | How is the token count computed?                                                       | Each element is encoded using the model's token encoding, and the length of the token list is returned.                                                                                                                                                                                                                                                                                                                                                                                  | [ 0.00427531 -0.00177265  0.04057357 ... -0.01623502 -0.00966137        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.04381499]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.extract      | function            | Expands a Series of Pydantic models or dictionaries into a DataFrame with one column per key. If the Series has a name, the extracted columns are prefixed with it.                                                                                                                                                                                                                                                                                  | What does this method return?                                                          | It returns a DataFrame with the same index as the Series, where each column corresponds to a key in the dict or BaseModel.                                                                                                                                                                                                                                                                                                                                                               | [-0.02367564  0.0244071   0.05108324 ... -0.00437387 -0.00980762        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01958539]                                                           |
| openaivec.pandas_ext     | OpenAIVecSeriesAccessor.extract      | function            | Expands a Series of Pydantic models or dictionaries into a DataFrame with one column per key. If the Series has a name, the extracted columns are prefixed with it.                                                                                                                                                                                                                                                                                  | How are column names handled if the Series has a name?                                 | Extracted columns are prefixed with the Series name followed by an underscore.                                                                                                                                                                                                                                                                                                                                                                                                           | [ 0.03439854 -0.02008603  0.0189095  ...  0.00992779  0.02366416        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00933346]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor           | class               | A pandas DataFrame accessor registered as '.ai' that adds OpenAI-related helper methods to pandas DataFrame objects. It enables extracting nested dict/BaseModel columns and generating LLM responses from rows.                                                                                                                                                                                                                                     | How do you access the OpenAI helpers on a pandas DataFrame?                            | By using the '.ai' accessor, e.g., 'df.ai.responses(...)'.                                                                                                                                                                                                                                                                                                                                                                                                                               | [-0.01114788 -0.01677558  0.07288537 ...  0.01141074 -0.01145854        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01147048]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor           | class               | A pandas DataFrame accessor registered as '.ai' that adds OpenAI-related helper methods to pandas DataFrame objects. It enables extracting nested dict/BaseModel columns and generating LLM responses from rows.                                                                                                                                                                                                                                     | What methods does this accessor provide?                                               | It provides 'extract' to flatten a column of dicts/BaseModels into top-level columns, and 'responses' to generate LLM responses for each row serialized as JSON.                                                                                                                                                                                                                                                                                                                         | [ 0.00432435  0.01770993  0.04900433 ... -0.0284508  -0.0166049         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0297179 ]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor.extract   | function            | Flattens one column of Pydantic models or dictionaries into separate top-level columns in the DataFrame, dropping the original column.                                                                                                                                                                                                                                                                                                               | What happens if the specified column does not exist?                                   | A ValueError is raised indicating the column does not exist.                                                                                                                                                                                                                                                                                                                                                                                                                             | [-0.01222972  0.05048428  0.04960374 ... -0.0057602   0.00627385        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03700713]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor.extract   | function            | Flattens one column of Pydantic models or dictionaries into separate top-level columns in the DataFrame, dropping the original column.                                                                                                                                                                                                                                                                                                               | Does the method preserve the original DataFrame index?                                 | Yes, the original index is preserved after extraction.                                                                                                                                                                                                                                                                                                                                                                                                                                   | [ 0.0176597  -0.03558936  0.00499702 ...  0.0195269   0.01552254        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00149742]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor.responses | function            | Generates a response for each DataFrame row by serializing the row to JSON and sending it to the OpenAI model with given instructions. Returns a Series of responses aligned with the DataFrame's index.                                                                                                                                                                                                                                             | How are rows sent to the OpenAI model?                                                 | Each row is converted to a JSON string and sent as a prompt to the model.                                                                                                                                                                                                                                                                                                                                                                                                                | [-0.00038698  0.01277437  0.02389145 ...  0.00860463 -0.00465035        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.02619839]                                                           |
| openaivec.pandas_ext     | OpenAIVecDataFrameAccessor.responses | function            | Generates a response for each DataFrame row by serializing the row to JSON and sending it to the OpenAI model with given instructions. Returns a Series of responses aligned with the DataFrame's index.                                                                                                                                                                                                                                             | What is the default response format?                                                   | The default response format is 'str', returning plain text responses.                                                                                                                                                                                                                                                                                                                                                                                                                    | [-0.06607127 -0.01000901  0.03518557 ... -0.00139369 -0.00021303        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01595411]                                                           |
| openaivec.spark          | UDFBuilder                           | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication details, batching behavior, and HTTP client configuration, allowing reuse across Spark worker nodes. The class provides factory methods to create UDFs for generating chat completions (responses) and embeddings, abstracting away low-level API details and enabling efficient processing of large data volumes. | What is the purpose of the UDFBuilder class?                                           | The UDFBuilder class is designed to create Spark pandas-UDFs that communicate with OpenAI or Azure OpenAI APIs. It manages API keys, model names, batching, and HTTP client settings to facilitate generating chat completions and embeddings within Spark DataFrames.                                                                                                                                                                                                                   | [-0.02941521  0.01529391  0.05124583 ... -0.00638078 -0.04129107        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01229376]                                                           |
| openaivec.spark          | UDFBuilder                           | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication details, batching behavior, and HTTP client configuration, allowing reuse across Spark worker nodes. The class provides factory methods to create UDFs for generating chat completions (responses) and embeddings, abstracting away low-level API details and enabling efficient processing of large data volumes. | How do you create an instance of UDFBuilder for Azure OpenAI?                          | Use the class method of_azureopenai, providing the API key, Azure endpoint, API version, model name, and optional parameters like batch size and HTTP settings.                                                                                                                                                                                                                                                                                                                          | [-0.04129735 -0.01343033  0.06788469 ... -0.02204016 -0.01017525        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0019956 ]                                                           |
| openaivec.spark          | UDFBuilder                           | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication details, batching behavior, and HTTP client configuration, allowing reuse across Spark worker nodes. The class provides factory methods to create UDFs for generating chat completions (responses) and embeddings, abstracting away low-level API details and enabling efficient processing of large data volumes. | How do you create an instance of UDFBuilder for the public OpenAI API?                 | Use the class method of_openai, providing the API key, model name, and optional parameters such as batch size and HTTP settings.                                                                                                                                                                                                                                                                                                                                                         | [-0.02859116 -0.01926141  0.05572769 ... -0.00993794 -0.02831528        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00562418]                                                           |
| openaivec.spark          | UDFBuilder                           | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication details, batching behavior, and HTTP client configuration, allowing reuse across Spark worker nodes. The class provides factory methods to create UDFs for generating chat completions (responses) and embeddings, abstracting away low-level API details and enabling efficient processing of large data volumes. | What does the responses method do?                                                     | The responses method returns a pandas UDF that produces chat completions. It accepts system instructions, a response format (either a string or a Pydantic model), and sampling parameters like temperature and top_p. The UDF outputs either a string column or a structured column matching the Pydantic model schema.                                                                                                                                                                 | [-0.03914329  0.02508372  0.06850075 ...  0.00845688 -0.01349326        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00346581]                                                           |
| openaivec.spark          | UDFBuilder                           | class               | A builder class for creating Spark pandas-UDFs that interact with the OpenAI or Azure OpenAI APIs. It encapsulates authentication details, batching behavior, and HTTP client configuration, allowing reuse across Spark worker nodes. The class provides factory methods to create UDFs for generating chat completions (responses) and embeddings, abstracting away low-level API details and enabling efficient processing of large data volumes. | What does the embeddings method do?                                                    | The embeddings method returns a pandas UDF that generates embedding vectors for input strings. The output column is an array of floats representing the embeddings.                                                                                                                                                                                                                                                                                                                      | [-0.01767209 -0.02698541  0.03454442 ... -0.01858794 -0.02087113        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00695274]                                                           |
| openaivec.prompt         | Example                              | class               | Represents a single input/output example used in few‑shot prompts. It holds the input text and the expected output text that demonstrate the desired behavior for the model.                                                                                                                                                                                                                                                                         | What attributes does the Example class have?                                           | The Example class has two attributes: 'input' (a string representing the input text) and 'output' (a string representing the expected output).                                                                                                                                                                                                                                                                                                                                           | [-0.02705586  0.00236532  0.01096134 ... -0.00838787 -0.01483531        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01301186]                                                           |
| openaivec.prompt         | Example                              | class               | Represents a single input/output example used in few‑shot prompts. It holds the input text and the expected output text that demonstrate the desired behavior for the model.                                                                                                                                                                                                                                                                         | What is the purpose of the Example class?                                              | It encapsulates a single example pair of input and output to be used in few-shot learning prompts, helping the model understand the expected behavior.                                                                                                                                                                                                                                                                                                                                   | [-0.01209102 -0.00847259 -0.02818279 ...  0.00026477 -0.03880596        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.0093533 ]                                                           |
| openaivec.prompt         | FewShotPrompt                        | class               | Represents a complete few-shot prompt definition, including the purpose, cautions, and examples. This model is used to structure the prompt data before rendering it into XML for use with large language models.                                                                                                                                                                                                                                    | What fields are included in FewShotPrompt?                                             | FewShotPrompt includes 'purpose' (a string describing the prompt's goal), 'cautions' (a list of strings warning about edge cases or pitfalls), and 'examples' (a list of Example instances demonstrating input/output pairs).                                                                                                                                                                                                                                                            | [-0.00160104  0.00097476 -0.0050713  ... -0.02967625 -0.01111896        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01911613]                                                           |
| openaivec.prompt         | FewShotPrompt                        | class               | Represents a complete few-shot prompt definition, including the purpose, cautions, and examples. This model is used to structure the prompt data before rendering it into XML for use with large language models.                                                                                                                                                                                                                                    | How is FewShotPrompt used?                                                             | It collects all necessary information to define a few-shot prompt, which can then be serialized into XML and sent to an LLM as part of the system prompt.                                                                                                                                                                                                                                                                                                                                | [-0.00116741 -0.0026398  -0.02530478 ... -0.03917042 -0.04239685        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00858603]                                                           |
| openaivec.prompt         | Step                                 | class               | Represents a single refinement iteration of the prompt produced by the LLM. Each step includes an identifier, an analysis explaining the change, and the updated prompt after the modification.                                                                                                                                                                                                                                                      | What does the 'id' attribute in Step represent?                                        | It is a sequential identifier for the iteration, starting at 0 for the original prompt and incrementing for each refinement step.                                                                                                                                                                                                                                                                                                                                                        | [ 0.00081196  0.00397757 -0.04023483 ... -0.02107669 -0.00029956        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02661309]                                                           |
| openaivec.prompt         | Step                                 | class               | Represents a single refinement iteration of the prompt produced by the LLM. Each step includes an identifier, an analysis explaining the change, and the updated prompt after the modification.                                                                                                                                                                                                                                                      | What information does 'analysis' provide?                                              | It provides a natural-language explanation of the issue addressed in the iteration and why the change was necessary.                                                                                                                                                                                                                                                                                                                                                                     | [0.00803303 0.03793377 0.00246569 ... 0.01694375 0.00265908 0.05316678] |
| openaivec.prompt         | Step                                 | class               | Represents a single refinement iteration of the prompt produced by the LLM. Each step includes an identifier, an analysis explaining the change, and the updated prompt after the modification.                                                                                                                                                                                                                                                      | What is stored in 'prompt' within a Step?                                              | The updated FewShotPrompt instance after applying the described modification in that iteration.                                                                                                                                                                                                                                                                                                                                                                                          | [ 0.00573075  0.01724004 -0.03213553 ... -0.01442404 -0.00265035        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00517647]                                                           |
| openaivec.prompt         | Request                              | class               | A simple wrapper model containing a FewShotPrompt instance, used to structure the request sent to the LLM for prompt improvement.                                                                                                                                                                                                                                                                                                                    | What is the purpose of the Request class?                                              | It encapsulates the FewShotPrompt to be sent as input to the LLM for iterative refinement.                                                                                                                                                                                                                                                                                                                                                                                               | [-0.01176378  0.01586656 -0.01771064 ...  0.01891115 -0.00274911        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01329226]                                                           |
| openaivec.prompt         | Response                             | class               | Represents the response from the LLM containing a list of Step instances, each describing an iteration of prompt refinement.                                                                                                                                                                                                                                                                                                                         | What does the 'iterations' attribute contain?                                          | A list of Step objects representing each refinement iteration produced by the LLM.                                                                                                                                                                                                                                                                                                                                                                                                       | [-0.00163387  0.01362625  0.04830925 ... -0.01719903 -0.01052838        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02169971]                                                           |
| openaivec.prompt         | _render_prompt                       | function            | Converts a FewShotPrompt instance into its XML string representation. It creates XML elements for purpose, cautions, and examples, formatting them according to the expected schema.                                                                                                                                                                                                                                                                 | What XML structure does _render_prompt produce?                                        | It produces a <Prompt> root element containing <Purpose>, <Cautions> with multiple <Caution> elements, and <Examples> with multiple <Example> elements each having <Input> and <Output>.                                                                                                                                                                                                                                                                                                 | [ 0.0242124   0.05072603  0.03355341 ...  0.00251774 -0.04451518        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00868529]                                                           |
| openaivec.prompt         | _render_prompt                       | function            | Converts a FewShotPrompt instance into its XML string representation. It creates XML elements for purpose, cautions, and examples, formatting them according to the expected schema.                                                                                                                                                                                                                                                                 | Does _render_prompt handle empty cautions?                                             | Yes, it always outputs the <Cautions> element, but only adds <Caution> children if cautions are present.                                                                                                                                                                                                                                                                                                                                                                                 | [ 0.01673163  0.0140115   0.03830849 ...  0.00845082 -0.01525822        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00288305]                                                           |
| openaivec.prompt         | FewShotPromptBuilder                 | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt using an LLM. It supports outputting the prompt as XML or JSON.                                                                                                                                                                                                                          | How do you set the purpose of the prompt using FewShotPromptBuilder?                   | By calling the 'purpose' method with a string describing the prompt's goal, which returns the builder instance for chaining.                                                                                                                                                                                                                                                                                                                                                             | [ 0.0179185  -0.02749075 -0.00111328 ... -0.02671271 -0.04559787        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.03201753]                                                           |
| openaivec.prompt         | FewShotPromptBuilder                 | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt using an LLM. It supports outputting the prompt as XML or JSON.                                                                                                                                                                                                                          | How are cautions added?                                                                | Using the 'caution' method, which appends a caution string to the internal list.                                                                                                                                                                                                                                                                                                                                                                                                         | [ 4.8624769e-02  2.3317857e-02  4.1494768e-02 ... -1.1941986e-02        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -7.5269454e-06 -1.9125538e-02]                                         |
| openaivec.prompt         | FewShotPromptBuilder                 | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt using an LLM. It supports outputting the prompt as XML or JSON.                                                                                                                                                                                                                          | How do you add examples?                                                               | By calling the 'example' method with input and output values, which can be strings or Pydantic models; models are serialized to JSON.                                                                                                                                                                                                                                                                                                                                                    | [-0.01435151 -0.0267385   0.01598611 ... -0.01286687 -0.01224452        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01010754]                                                           |
| openaivec.prompt         | FewShotPromptBuilder                 | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt using an LLM. It supports outputting the prompt as XML or JSON.                                                                                                                                                                                                                          | What does the 'improve' method do?                                                     | It sends the current prompt to an LLM to iteratively refine it, receiving multiple editing steps, storing them, and updating the prompt to the final refined version. It requires at least 5 examples.                                                                                                                                                                                                                                                                                   | [-0.03149921  0.03178376  0.01076294 ... -0.01180865 -0.04897032        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01254847]                                                           |
| openaivec.prompt         | FewShotPromptBuilder                 | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt using an LLM. It supports outputting the prompt as XML or JSON.                                                                                                                                                                                                                          | What is the purpose of the 'explain' method?                                           | It pretty-prints the differences between each refinement iteration, showing the analysis and XML diff between prompt versions.                                                                                                                                                                                                                                                                                                                                                           | [-0.00277646  0.04553728 -0.02155956 ...  0.01092243 -0.02539056        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0107594 ]                                                           |
| openaivec.prompt         | FewShotPromptBuilder                 | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt using an LLM. It supports outputting the prompt as XML or JSON.                                                                                                                                                                                                                          | How does the builder validate the prompt?                                              | The '_validate' method ensures that the purpose is set and that there is at least one example; otherwise, it raises a ValueError.                                                                                                                                                                                                                                                                                                                                                        | [-0.00201246  0.01227503 -0.0004474  ... -0.03722563 -0.03442978        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.02345057]                                                           |
| openaivec.prompt         | FewShotPromptBuilder                 | class               | A builder class to construct and iteratively improve few-shot prompts. It allows setting the purpose, adding cautions and examples, and refining the prompt using an LLM. It supports outputting the prompt as XML or JSON.                                                                                                                                                                                                                          | How can the final prompt be obtained?                                                  | By calling 'build' or 'build_xml' to get the XML string, or 'build_json' to get the JSON representation.                                                                                                                                                                                                                                                                                                                                                                                 | [ 0.053101    0.00339733  0.01697516 ... -0.0060248   0.00226936        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01306076]                                                           |
| openaivec.beta.batch     | JsonlUDFBuilder                      | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines format for chat completion requests to a specified model.                                                                                                                                                                                                                                                                            | What is the purpose of the JsonlUDFBuilder class?                                      | The JsonlUDFBuilder class is designed to create a PySpark pandas UDF that transforms input data (custom IDs and user messages) into JSON Lines formatted strings. These strings represent HTTP POST requests to a chat completion API using a specified model.                                                                                                                                                                                                                           | [-0.02770036 -0.02106407  0.01810232 ... -0.02183831 -0.02531621        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00492499]                                                           |
| openaivec.beta.batch     | JsonlUDFBuilder                      | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines format for chat completion requests to a specified model.                                                                                                                                                                                                                                                                            | What does the 'model_name' attribute represent?                                        | The 'model_name' attribute stores the name of the model to be used in the chat completion requests within the generated JSON payload.                                                                                                                                                                                                                                                                                                                                                    | [0.00451574 0.03469364 0.00187493 ... 0.01953507 0.02021144 0.00621992] |
| openaivec.beta.batch     | JsonlUDFBuilder                      | class               | A frozen dataclass that encapsulates the building of a PySpark pandas UDF to convert input data into JSON Lines format for chat completion requests to a specified model.                                                                                                                                                                                                                                                                            | Why is the class marked as frozen?                                                     | Marking the class as frozen makes it immutable after instantiation, ensuring that the 'model_name' attribute cannot be changed, which helps maintain consistency and thread safety.                                                                                                                                                                                                                                                                                                      | [-0.01471578  0.03379414  0.01319837 ...  0.01439175  0.02393093        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00078736]                                                           |
| openaivec.beta.batch     | to_jsonl                             | function            | A method that returns a PySpark pandas UDF. This UDF takes iterators of pandas Series for custom IDs and user messages, and yields pandas Series of JSON strings formatted as chat completion requests with specified parameters.                                                                                                                                                                                                                    | What parameters does the to_jsonl method accept and what are their roles?              | The method accepts 'system_message' (a string to be included as the system role message), 'temperature' (a float controlling randomness in the model's output, default 0.0), and 'top_p' (a float controlling nucleus sampling, default 1.0). These parameters configure the chat completion request payload.                                                                                                                                                                            | [-0.01128534 -0.02550451  0.01540557 ... -0.00203235  0.01445879        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00599625]                                                           |
| openaivec.beta.batch     | to_jsonl                             | function            | A method that returns a PySpark pandas UDF. This UDF takes iterators of pandas Series for custom IDs and user messages, and yields pandas Series of JSON strings formatted as chat completion requests with specified parameters.                                                                                                                                                                                                                    | How does the returned UDF process the input data?                                      | The UDF iterates over pairs of 'custom_id' and 'user_message' pandas Series. For each pair, it creates a DataFrame, then applies a function row-wise to generate a JSON string representing a POST request with the model name, system message, user message, temperature, and top_p parameters. It yields these JSON strings as a pandas Series.                                                                                                                                        | [-0.03840687  0.04318549  0.06532472 ... -0.014679   -0.01409438        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02244425]                                                           |
| openaivec.beta.batch     | to_jsonl                             | function            | A method that returns a PySpark pandas UDF. This UDF takes iterators of pandas Series for custom IDs and user messages, and yields pandas Series of JSON strings formatted as chat completion requests with specified parameters.                                                                                                                                                                                                                    | What is the structure of the JSON produced by the UDF?                                 | Each JSON string includes 'custom_id', HTTP 'method' set to 'POST', 'url' set to '/chat/completions', and a 'body' containing the 'model' name, a 'messages' list with system and user roles and their contents, and the 'temperature' and 'top_p' parameters.                                                                                                                                                                                                                           | [-0.03679636  0.00538902  0.03644687 ... -0.00705846 -0.0124194         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01997089]                                                           |
| openaivec.beta.batch     | to_jsonl                             | function            | A method that returns a PySpark pandas UDF. This UDF takes iterators of pandas Series for custom IDs and user messages, and yields pandas Series of JSON strings formatted as chat completion requests with specified parameters.                                                                                                                                                                                                                    | Why is pandas_udf used here?                                                           | pandas_udf is used to define a vectorized user-defined function in PySpark that operates on pandas Series, enabling efficient processing of large datasets in a distributed manner while producing JSON Lines formatted output.                                                                                                                                                                                                                                                          | [-0.02388386 -0.02058294  0.06622227 ... -0.00132785 -0.00997922        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02463581]                                                           |
| openaivec.beta.schema    | Entity                               | class               | Represents an entity extracted from text, with its textual content and schema.org classification.                                                                                                                                                                                                                                                                                                                                                    | What attributes does the Entity class have?                                            | The Entity class has two attributes: 'body', a string representing the text of the entity, and 'entity_class', a string representing the schema.org class hierarchy of the entity (e.g., 'Thing.Person').                                                                                                                                                                                                                                                                                | [-0.03689022  0.03209228  0.01593599 ... -0.01883678 -0.00939391        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00484995]                                                           |
| openaivec.beta.schema    | Entity                               | class               | Represents an entity extracted from text, with its textual content and schema.org classification.                                                                                                                                                                                                                                                                                                                                                    | What is the purpose of the Entity class?                                               | The Entity class models an individual entity identified in the input text, including its textual representation and its classification according to schema.org ontology.                                                                                                                                                                                                                                                                                                                 | [-0.02242264  0.02215512 -0.00375433 ... -0.00471496 -0.02940235        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00315242]                                                           |
| openaivec.beta.schema    | Relationship                         | class               | Models a relationship between two entities, specifying the tail and head entities and the property that connects them.                                                                                                                                                                                                                                                                                                                               | What attributes are defined in the Relationship class?                                 | The Relationship class has three attributes: 'tail' and 'head', both of type Entity representing the two ends of the relationship, and 'property', a string indicating the schema.org property that defines the relationship.                                                                                                                                                                                                                                                            | [-0.03552138  0.02256667  0.02566975 ...  0.02112866  0.02463539        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00442756]                                                           |
| openaivec.beta.schema    | Relationship                         | class               | Models a relationship between two entities, specifying the tail and head entities and the property that connects them.                                                                                                                                                                                                                                                                                                                               | What does the Relationship class represent?                                            | It represents a directed relationship from the 'tail' entity to the 'head' entity, characterized by a schema.org property.                                                                                                                                                                                                                                                                                                                                                               | [-0.02410616  0.01241625  0.01079227 ...  0.0377421   0.00424838        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01491046]                                                           |
| openaivec.beta.schema    | RelationshipResponse                 | class               | Encapsulates a list of Relationship objects as the response output of the extraction process.                                                                                                                                                                                                                                                                                                                                                        | What is the purpose of the RelationshipResponse class?                                 | It serves as a container for multiple Relationship instances, representing all extracted relationships from the input text.                                                                                                                                                                                                                                                                                                                                                              | [-0.0352179   0.0305944  -0.00476221 ...  0.0346895  -0.00064069        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01064727]                                                           |
| openaivec.beta.schema    | RelationshipResponse                 | class               | Encapsulates a list of Relationship objects as the response output of the extraction process.                                                                                                                                                                                                                                                                                                                                                        | What attribute does RelationshipResponse have?                                         | It has a single attribute 'relationships', which is a list of Relationship objects.                                                                                                                                                                                                                                                                                                                                                                                                      | [-0.04845317  0.02878025 -0.0041547  ...  0.02197039  0.0125948         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00173342]                                                           |
| openaivec.beta.schema    | schemaorg_extraction_prompt          | variable            | A detailed prompt string guiding the extraction of entities and relationships from text according to schema.org standards.                                                                                                                                                                                                                                                                                                                           | What is the purpose of the schemaorg_extraction_prompt variable?                       | It provides instructions and an example for analyzing text to extract entities and their relationships strictly following schema.org classes and properties, formatted as a JSON object.                                                                                                                                                                                                                                                                                                 | [ 0.04621952  0.00197485  0.0364298  ... -0.00871311 -0.04073628        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00304208]                                                           |
| openaivec.beta.schema    | schemaorg_extraction_prompt          | variable            | A detailed prompt string guiding the extraction of entities and relationships from text according to schema.org standards.                                                                                                                                                                                                                                                                                                                           | What key instructions does the prompt include?                                         | The prompt instructs to identify all entities, classify them with full schema.org hierarchy notation, identify relationships using only schema.org properties, and return a JSON object with no extra text. It also forbids inventing custom classes or properties.                                                                                                                                                                                                                      | [ 0.04420609  0.01671933  0.02590999 ... -0.02900669  0.00979437        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0095742 ]                                                           |
| openaivec.beta.schema    | schemaorg_extraction_prompt          | variable            | A detailed prompt string guiding the extraction of entities and relationships from text according to schema.org standards.                                                                                                                                                                                                                                                                                                                           | What example does the prompt provide?                                                  | It shows an example input sentence about an Oscars broadcast event and the corresponding JSON output with entities classified as 'Thing.Event.PublicationEvent.BroadcastEvent' and 'Thing.Place.LandmarksOrHistoricalBuildings', and relationships like 'location' and 'organizer'.                                                                                                                                                                                                      | [ 0.02372681 -0.00148107 -0.01001137 ... -0.01484599 -0.01615506        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.0022295 ]                                                           |
| openaivec.aio.responses  | AsyncBatchResponses                  | class               | A stateless asynchronous wrapper that converts OpenAI's JSON-mode API into a batched API. It allows submitting multiple user prompts in a single JSON request and receiving responses asynchronously in the original order. It also manages concurrency limits for requests to the OpenAI API.                                                                                                                                                       | What is the purpose of the AsyncBatchResponses class?                                  | It provides an asynchronous interface to batch multiple prompts into a single JSON-mode request to the OpenAI API, returning responses in the original order while controlling concurrency.                                                                                                                                                                                                                                                                                              | [-0.02701208 -0.00284884  0.01162987 ...  0.00580591 -0.04370521        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00695747]                                                           |
| openaivec.aio.responses  | AsyncBatchResponses                  | class               | A stateless asynchronous wrapper that converts OpenAI's JSON-mode API into a batched API. It allows submitting multiple user prompts in a single JSON request and receiving responses asynchronously in the original order. It also manages concurrency limits for requests to the OpenAI API.                                                                                                                                                       | What are the main attributes of AsyncBatchResponses?                                   | Attributes include the OpenAI async client, model name, system message, temperature, top_p, response format type, maximum concurrency, and internal fields for vectorized system message, model schema, and a semaphore for concurrency control.                                                                                                                                                                                                                                         | [-0.03053455  0.00185758  0.01825777 ... -0.01800857 -0.02456667        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00085501]                                                           |
| openaivec.aio.responses  | AsyncBatchResponses                  | class               | A stateless asynchronous wrapper that converts OpenAI's JSON-mode API into a batched API. It allows submitting multiple user prompts in a single JSON request and receiving responses asynchronously in the original order. It also manages concurrency limits for requests to the OpenAI API.                                                                                                                                                       | How does AsyncBatchResponses handle concurrency?                                       | It uses an asyncio.Semaphore initialized with max_concurrency to limit the number of concurrent requests to the OpenAI API.                                                                                                                                                                                                                                                                                                                                                              | [-0.02673371 -0.04087197  0.05086234 ... -0.01499819 -0.01297483        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00205498]                                                           |
| openaivec.aio.responses  | AsyncBatchResponses                  | class               | A stateless asynchronous wrapper that converts OpenAI's JSON-mode API into a batched API. It allows submitting multiple user prompts in a single JSON request and receiving responses asynchronously in the original order. It also manages concurrency limits for requests to the OpenAI API.                                                                                                                                                       | What is the role of the _vectorized_system_message attribute?                          | It stores a processed version of the system message, prepared by the _vectorize_system_message function, to be used as instructions in API calls.                                                                                                                                                                                                                                                                                                                                        | [ 0.02651847  0.04943841 -0.05785345 ...  0.00849809 -0.00271967        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03952858]                                                           |
| openaivec.aio.responses  | __post_init__                        | function            | Post-initialization method for the dataclass that sets up the vectorized system message and initializes the concurrency semaphore. Uses object.__setattr__ because the dataclass is frozen.                                                                                                                                                                                                                                                          | Why is object.__setattr__ used in __post_init__?                                       | Because the dataclass is frozen (immutable), direct attribute assignment is not allowed, so object.__setattr__ is used to bypass immutability for initialization.                                                                                                                                                                                                                                                                                                                        | [ 0.02520901  0.03321977 -0.03344606 ...  0.00196592  0.02441698        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.03355921]                                                           |
| openaivec.aio.responses  | __post_init__                        | function            | Post-initialization method for the dataclass that sets up the vectorized system message and initializes the concurrency semaphore. Uses object.__setattr__ because the dataclass is frozen.                                                                                                                                                                                                                                                          | What does __post_init__ initialize?                                                    | It initializes the _vectorized_system_message by processing the system_message and creates an asyncio.Semaphore to control max concurrent requests.                                                                                                                                                                                                                                                                                                                                      | [-0.02707829  0.00427621  0.00264796 ... -0.00097859  0.00432555        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01233523]                                                           |
| openaivec.aio.responses  | _request_llm                         | function            | Makes a single asynchronous call to the OpenAI JSON-mode endpoint with a batch of user messages, respecting concurrency limits and handling rate limiting with exponential backoff. Returns parsed responses containing assistant messages.                                                                                                                                                                                                          | What does _request_llm do?                                                             | It sends a batch of user messages to the OpenAI API asynchronously, using a semaphore to limit concurrency, and returns the parsed response containing assistant messages.                                                                                                                                                                                                                                                                                                               | [-0.0135437   0.00213964  0.02233754 ... -0.00586379  0.0026194         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03464598]                                                           |
| openaivec.aio.responses  | _request_llm                         | function            | Makes a single asynchronous call to the OpenAI JSON-mode endpoint with a batch of user messages, respecting concurrency limits and handling rate limiting with exponential backoff. Returns parsed responses containing assistant messages.                                                                                                                                                                                                          | How does _request_llm handle rate limiting?                                            | It is decorated with a backoff decorator that retries on RateLimitError with exponential backoff, up to a maximum number of retries.                                                                                                                                                                                                                                                                                                                                                     | [ 0.00347176 -0.01547884  0.03515673 ... -0.01812726  0.01575329        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02487867]                                                           |
| openaivec.aio.responses  | _request_llm                         | function            | Makes a single asynchronous call to the OpenAI JSON-mode endpoint with a batch of user messages, respecting concurrency limits and handling rate limiting with exponential backoff. Returns parsed responses containing assistant messages.                                                                                                                                                                                                          | What is the purpose of the nested MessageT and ResponseT classes?                      | They define Pydantic models for the expected response format, where MessageT represents an assistant message with an id and body, and ResponseT wraps a list of such messages.                                                                                                                                                                                                                                                                                                           | [-0.0330671   0.01225973  0.03287172 ...  0.01230247 -0.00311836        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01484844]                                                           |
| openaivec.aio.responses  | _request_llm                         | function            | Makes a single asynchronous call to the OpenAI JSON-mode endpoint with a batch of user messages, respecting concurrency limits and handling rate limiting with exponential backoff. Returns parsed responses containing assistant messages.                                                                                                                                                                                                          | How does _request_llm ensure concurrency limits?                                       | It uses an async with block on self._semaphore to acquire a semaphore slot before making the API call, ensuring no more than max_concurrency calls run simultaneously.                                                                                                                                                                                                                                                                                                                   | [ 0.01823352 -0.01332664  0.0653507  ... -0.02008753  0.00623466        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02535685]                                                           |
| openaivec.aio.responses  | _predict_chunk                       | function            | Helper asynchronous method that processes a minibatch of user prompt strings. It converts prompts into Message objects with stable indices, calls _request_llm to get responses, and reorders the responses to match the original prompt order. The method is pure and side-effect free.                                                                                                                                                             | What is the input and output of _predict_chunk?                                        | Input is a list of user prompt strings; output is a list of responses of type T, ordered to correspond to the input prompts.                                                                                                                                                                                                                                                                                                                                                             | [ 0.03298789 -0.02031054 -0.0091788  ... -0.02577876 -0.00438853        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00162512]                                                           |
| openaivec.aio.responses  | _predict_chunk                       | function            | Helper asynchronous method that processes a minibatch of user prompt strings. It converts prompts into Message objects with stable indices, calls _request_llm to get responses, and reorders the responses to match the original prompt order. The method is pure and side-effect free.                                                                                                                                                             | How does _predict_chunk maintain the order of responses?                               | It assigns each prompt an id, sends them as Message objects, then reconstructs the response list by matching response ids to the original message ids.                                                                                                                                                                                                                                                                                                                                   | [ 0.00508926 -0.02836204  0.00345248 ... -0.01884618  0.01084119        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01391596]                                                           |
| openaivec.aio.responses  | _predict_chunk                       | function            | Helper asynchronous method that processes a minibatch of user prompt strings. It converts prompts into Message objects with stable indices, calls _request_llm to get responses, and reorders the responses to match the original prompt order. The method is pure and side-effect free.                                                                                                                                                             | Is _predict_chunk stateful?                                                            | No, it is a pure function with no side effects; the output depends only on its input.                                                                                                                                                                                                                                                                                                                                                                                                    | [ 0.01065913 -0.02122874 -0.0356029  ... -0.03378695  0.00582192        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00307081]                                                           |
| openaivec.aio.responses  | parse                                | function            | Public asynchronous method to perform batched prediction on a list of input prompts. It de-duplicates inputs internally and processes them in batches of a specified size, returning responses in the same order as the inputs.                                                                                                                                                                                                                      | What arguments does parse accept?                                                      | A list of input prompt strings and an integer batch_size specifying the maximum number of unique prompts per API call.                                                                                                                                                                                                                                                                                                                                                                   | [ 0.00528061  0.00839949 -0.00064502 ...  0.00668704 -0.00319702        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00237009]                                                           |
| openaivec.aio.responses  | parse                                | function            | Public asynchronous method to perform batched prediction on a list of input prompts. It de-duplicates inputs internally and processes them in batches of a specified size, returning responses in the same order as the inputs.                                                                                                                                                                                                                      | How does parse handle batching?                                                        | It uses an asynchronous map utility to split inputs into batches and calls _predict_chunk on each batch asynchronously.                                                                                                                                                                                                                                                                                                                                                                  | [-0.0272127  -0.04015828  0.01937792 ... -0.01488742 -0.01615501        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01593925]                                                           |
| openaivec.aio.responses  | parse                                | function            | Public asynchronous method to perform batched prediction on a list of input prompts. It de-duplicates inputs internally and processes them in batches of a specified size, returning responses in the same order as the inputs.                                                                                                                                                                                                                      | Does parse preserve the order of responses?                                            | Yes, it returns a list of responses aligned with the order of the original inputs.                                                                                                                                                                                                                                                                                                                                                                                                       | [-0.01750349 -0.01276669  0.00266059 ... -0.01301418  0.02402089        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00180122]                                                           |
| openaivec.aio.util       | map                                  | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                                                                               | What is the purpose of the `map` function?                                             | The `map` function applies an asynchronous function `f` to a list of inputs in batches, allowing concurrent processing of these batches. It returns the results in the same order as the original inputs.                                                                                                                                                                                                                                                                                | [ 0.01260459  0.04406585  0.0378891  ...  0.01397302 -0.03299289        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00531992]                                                           |
| openaivec.aio.util       | map                                  | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                                                                               | How does the function handle duplicate inputs?                                         | It computes hashes of the inputs to identify unique inputs, processes only these unique inputs in batches, and then reconstructs the output list by mapping the results back to the original input order using the hashes.                                                                                                                                                                                                                                                               | [ 0.04750434  0.00110031  0.03010147 ... -0.00736303  0.00085454        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01499302]                                                           |
| openaivec.aio.util       | map                                  | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                                                                               | Why are inputs hashed using `hash(str(v))` instead of just `hash(v)`?                  | Using `hash(str(v))` ensures that inputs which may not be directly hashable (like lists or dicts) can still be hashed by converting them to their string representation first.                                                                                                                                                                                                                                                                                                           | [-0.00756841  0.02548524  0.00945768 ... -0.03662794 -0.02086703        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00134319]                                                           |
| openaivec.aio.util       | map                                  | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                                                                               | How are batches created and processed?                                                 | The unique inputs are split into batches of size `batch_size`. Each batch is passed to the asynchronous function `f` concurrently using `asyncio.gather` to run all batch tasks in parallel.                                                                                                                                                                                                                                                                                             | [-0.02292813 -0.00612326  0.06713334 ... -0.06282625 -0.02312913        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00340799]                                                           |
| openaivec.aio.util       | map                                  | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                                                                               | What does the function expect from the asynchronous function `f`?                      | The function `f` should accept a list of inputs (a batch) and return an awaitable that resolves to a list of outputs of the same length, corresponding to each input in the batch.                                                                                                                                                                                                                                                                                                       | [-0.01045047  0.02234764  0.00223508 ...  0.01019032 -0.02314712        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.05164956]                                                           |
| openaivec.aio.util       | map                                  | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                                                                               | What happens if the number of outputs does not match the number of unique inputs?      | The function raises a `ValueError` indicating a mismatch, suggesting that the asynchronous function `f` did not return the expected number of outputs.                                                                                                                                                                                                                                                                                                                                   | [ 0.00725854 -0.01790047  0.04419383 ...  0.00243427 -0.02018229        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03559111]                                                           |
| openaivec.aio.util       | map                                  | function            | Asynchronously maps an asynchronous function `f` over a list of inputs in batches, preserving the order of the original inputs. It handles duplicate inputs by processing only unique inputs and then reconstructing the full output list accordingly.                                                                                                                                                                                               | How does the function ensure the output list matches the order of the original inputs? | It uses the hashes of the original inputs to look up the corresponding output from the unique outputs dictionary, reconstructing the full output list in the original input order.                                                                                                                                                                                                                                                                                                       | [ 0.0158798   0.00263173  0.01805399 ...  0.00308583 -0.01966398        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01828792]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings                 | class               | A dataclass providing an asynchronous interface to generate embeddings from the OpenAI embeddings endpoint. It manages concurrency with a semaphore, handles rate limiting with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication. The class uses an AsyncOpenAI client to perform the embedding requests and returns numpy float32 arrays for each input string's embedding.               | What is the purpose of the AsyncBatchEmbeddings class?                                 | It provides an asynchronous wrapper around the OpenAI embeddings API, allowing efficient batch processing of input strings with concurrency control and automatic handling of rate limits.                                                                                                                                                                                                                                                                                               | [-0.00124631 -0.02551957  0.00486776 ... -0.00220495 -0.04848478        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01309716]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings                 | class               | A dataclass providing an asynchronous interface to generate embeddings from the OpenAI embeddings endpoint. It manages concurrency with a semaphore, handles rate limiting with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication. The class uses an AsyncOpenAI client to perform the embedding requests and returns numpy float32 arrays for each input string's embedding.               | How does AsyncBatchEmbeddings handle concurrency?                                      | It uses an asyncio.Semaphore initialized with max_concurrency to limit the number of concurrent API requests.                                                                                                                                                                                                                                                                                                                                                                            | [ 0.00505686 -0.05115991  0.04018662 ... -0.01964219 -0.02918895        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00139605]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings                 | class               | A dataclass providing an asynchronous interface to generate embeddings from the OpenAI embeddings endpoint. It manages concurrency with a semaphore, handles rate limiting with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication. The class uses an AsyncOpenAI client to perform the embedding requests and returns numpy float32 arrays for each input string's embedding.               | What is the role of the _embed_chunk method?                                           | It is a private asynchronous method that sends a batch of input strings to the OpenAI embeddings endpoint, respecting concurrency limits and applying exponential backoff on rate limit errors.                                                                                                                                                                                                                                                                                          | [ 0.0017356   0.0064712  -0.02349738 ... -0.03912879 -0.03073507        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.01120209]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings                 | class               | A dataclass providing an asynchronous interface to generate embeddings from the OpenAI embeddings endpoint. It manages concurrency with a semaphore, handles rate limiting with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication. The class uses an AsyncOpenAI client to perform the embedding requests and returns numpy float32 arrays for each input string's embedding.               | How does the class handle duplicate inputs?                                            | The public create method uses an external utility (openaivec.aio.util.map) which efficiently handles batching and de-duplication, ensuring duplicate inputs are embedded only once and results are reused.                                                                                                                                                                                                                                                                               | [ 0.04447343  0.00583695  0.05890285 ...  0.01148132  0.00773322        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00539251]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings                 | class               | A dataclass providing an asynchronous interface to generate embeddings from the OpenAI embeddings endpoint. It manages concurrency with a semaphore, handles rate limiting with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication. The class uses an AsyncOpenAI client to perform the embedding requests and returns numpy float32 arrays for each input string's embedding.               | What does the create method return?                                                    | It returns a list of numpy.ndarray objects (dtype float32), each corresponding to the embedding of the input strings in the original order.                                                                                                                                                                                                                                                                                                                                              | [ 0.02701954  0.03896315  0.01680763 ... -0.02606456 -0.00296044        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00395998]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings                 | class               | A dataclass providing an asynchronous interface to generate embeddings from the OpenAI embeddings endpoint. It manages concurrency with a semaphore, handles rate limiting with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication. The class uses an AsyncOpenAI client to perform the embedding requests and returns numpy float32 arrays for each input string's embedding.               | What exceptions can be raised by the embedding methods?                                | openai.RateLimitError can be raised if the maximum number of retries for rate limiting is exhausted during API calls.                                                                                                                                                                                                                                                                                                                                                                    | [-0.00073057  0.0200445   0.07106301 ... -0.03387168 -0.03007378        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00294338]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings                 | class               | A dataclass providing an asynchronous interface to generate embeddings from the OpenAI embeddings endpoint. It manages concurrency with a semaphore, handles rate limiting with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication. The class uses an AsyncOpenAI client to perform the embedding requests and returns numpy float32 arrays for each input string's embedding.               | What is the significance of the @observe and @backoff decorators?                      | @observe logs or monitors the execution of the decorated methods using the provided logger. @backoff applies an exponential backoff retry strategy when a RateLimitError is encountered, with scaling and maximum retry limits.                                                                                                                                                                                                                                                          | [ 0.06125244 -0.01446853  0.00809961 ... -0.03095851 -0.01942522        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01478698]                                                           |
| openaivec.aio.embeddings | AsyncBatchEmbeddings                 | class               | A dataclass providing an asynchronous interface to generate embeddings from the OpenAI embeddings endpoint. It manages concurrency with a semaphore, handles rate limiting with automatic exponential backoff, and efficiently processes batches of input strings including de-duplication. The class uses an AsyncOpenAI client to perform the embedding requests and returns numpy float32 arrays for each input string's embedding.               | How is the semaphore initialized given the dataclass is frozen?                        | The semaphore is initialized in the __post_init__ method using object.__setattr__ to bypass the frozen restriction.                                                                                                                                                                                                                                                                                                                                                                      | [ 0.00706383  0.02779911  0.02342837 ...  0.02704858 -0.01645284        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.02239823]                                                           |
| openaivec.aio.pandas_ext | use                                  | function            | Registers a custom asynchronous OpenAI-compatible client to be used by the pandas extension helpers. The client must be an instance of either AsyncOpenAI or AsyncAzureOpenAI. This client is stored globally and reused for all OpenAI operations within the module.                                                                                                                                                                                | What types of clients can be registered using the use function?                        | Only instances of openai.AsyncOpenAI or openai.AsyncAzureOpenAI can be registered. If a different type is passed, a TypeError is raised.                                                                                                                                                                                                                                                                                                                                                 | [ 0.00191939  0.04459586  0.03205833 ... -0.02638274 -0.02551994        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0113377 ]                                                           |
| openaivec.aio.pandas_ext | use                                  | function            | Registers a custom asynchronous OpenAI-compatible client to be used by the pandas extension helpers. The client must be an instance of either AsyncOpenAI or AsyncAzureOpenAI. This client is stored globally and reused for all OpenAI operations within the module.                                                                                                                                                                                | Why is the client stored globally?                                                     | Storing the client globally allows all helper functions and accessors in the module to reuse the same OpenAI client instance, ensuring consistent configuration and avoiding repeated instantiation.                                                                                                                                                                                                                                                                                     | [-0.00878868 -0.00089859  0.04763272 ... -0.02047038  0.00507011        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02543821]                                                           |
| openaivec.aio.pandas_ext | use_openai                           | function            | Creates and registers a default AsyncOpenAI client using a provided API key. This function simplifies client setup for standard OpenAI usage by internally creating an AsyncOpenAI instance and storing it globally for reuse.                                                                                                                                                                                                                       | What parameter does use_openai require?                                                | It requires an api_key string which is forwarded to the AsyncOpenAI client constructor.                                                                                                                                                                                                                                                                                                                                                                                                  | [ 0.00113974  0.02567045  0.04965121 ...  0.01264676 -0.01832675        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00303821]                                                           |
| openaivec.aio.pandas_ext | use_openai                           | function            | Creates and registers a default AsyncOpenAI client using a provided API key. This function simplifies client setup for standard OpenAI usage by internally creating an AsyncOpenAI instance and storing it globally for reuse.                                                                                                                                                                                                                       | How does this function affect the global client?                                       | It sets the global _CLIENT variable to the newly created AsyncOpenAI instance, making it the default client for subsequent operations.                                                                                                                                                                                                                                                                                                                                                   | [-0.00967318  0.02228783  0.03635126 ... -0.01546831 -0.03342442        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03708296]                                                           |
| openaivec.aio.pandas_ext | use_azure_openai                     | function            | Creates and registers an AsyncAzureOpenAI client configured with Azure-specific parameters: API key, endpoint URL, and API version. This function enables integration with Azure OpenAI services by setting up the client accordingly and storing it globally.                                                                                                                                                                                       | What parameters are required to use Azure OpenAI?                                      | The function requires api_key (Azure subscription key), endpoint (resource endpoint URL), and api_version (REST API version string).                                                                                                                                                                                                                                                                                                                                                     | [-0.01777265  0.01175568  0.0652035  ... -0.013481    0.02614571        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.0082432 ]                                                           |
| openaivec.aio.pandas_ext | use_azure_openai                     | function            | Creates and registers an AsyncAzureOpenAI client configured with Azure-specific parameters: API key, endpoint URL, and API version. This function enables integration with Azure OpenAI services by setting up the client accordingly and storing it globally.                                                                                                                                                                                       | What does this function do with the created client?                                    | It assigns the created AsyncAzureOpenAI client to the global _CLIENT variable for reuse in the module.                                                                                                                                                                                                                                                                                                                                                                                   | [-0.01054387  0.00893128  0.02822726 ... -0.04746811 -0.04559364        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02877857]                                                           |
| openaivec.aio.pandas_ext | responses_model                      | function            | Overrides the model used for generating text responses from the OpenAI API. It updates the global model name and attempts to set the appropriate tiktoken encoding for token counting. If the model name is not recognized by tiktoken, it falls back to a default encoding and logs a warning.                                                                                                                                                      | What is the default model name before calling this function?                           | The default model name is 'gpt-4o-mini'.                                                                                                                                                                                                                                                                                                                                                                                                                                                 | [ 0.00500873  0.01008811 -0.0050052  ...  0.01188956 -0.00193567        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.00224828]                                                           |
| openaivec.aio.pandas_ext | responses_model                      | function            | Overrides the model used for generating text responses from the OpenAI API. It updates the global model name and attempts to set the appropriate tiktoken encoding for token counting. If the model name is not recognized by tiktoken, it falls back to a default encoding and logs a warning.                                                                                                                                                      | What happens if the provided model name is not supported by tiktoken?                  | A warning is logged, and the encoding falls back to 'o200k_base' to ensure token counting can still proceed.                                                                                                                                                                                                                                                                                                                                                                             | [-0.00290421 -0.00366881  0.00401493 ...  0.02695916  0.00590912        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03264803]                                                           |
| openaivec.aio.pandas_ext | embeddings_model                     | function            | Overrides the model used for generating text embeddings from the OpenAI API. It updates the global embedding model name used by the embeddings helper functions.                                                                                                                                                                                                                                                                                     | What is the default embedding model before calling this function?                      | The default embedding model is 'text-embedding-3-small'.                                                                                                                                                                                                                                                                                                                                                                                                                                 | [-0.00418682 -0.00327106  0.02483028 ... -0.01108374 -0.01670671        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.0002414 ]                                                           |
| openaivec.aio.pandas_ext | embeddings_model                     | function            | Overrides the model used for generating text embeddings from the OpenAI API. It updates the global embedding model name used by the embeddings helper functions.                                                                                                                                                                                                                                                                                     | Does this function affect token encoding?                                              | No, it only updates the embedding model name; token encoding is related to the response model.                                                                                                                                                                                                                                                                                                                                                                                           | [ 0.03029718 -0.02719611  0.00556066 ... -0.00430514 -0.02096653        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0232443 ]                                                           |
| openaivec.aio.pandas_ext | _get_openai_client                   | function            | Retrieves the currently registered OpenAI client. If no client is registered, it attempts to create one from environment variables for either OpenAI or Azure OpenAI. If no suitable credentials are found, it raises a ValueError instructing the user to set environment variables or register a client manually.                                                                                                                                  | What environment variables does this function check for OpenAI?                        | It checks for 'OPENAI_API_KEY' to create a standard AsyncOpenAI client.                                                                                                                                                                                                                                                                                                                                                                                                                  | [-0.0185667   0.01727197  0.0612928  ... -0.01867348 -0.01653785        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00518893]                                                           |
| openaivec.aio.pandas_ext | _get_openai_client                   | function            | Retrieves the currently registered OpenAI client. If no client is registered, it attempts to create one from environment variables for either OpenAI or Azure OpenAI. If no suitable credentials are found, it raises a ValueError instructing the user to set environment variables or register a client manually.                                                                                                                                  | What environment variables does it check for Azure OpenAI?                             | It checks for 'AZURE_OPENAI_API_KEY', 'AZURE_OPENAI_ENDPOINT', and 'AZURE_OPENAI_API_VERSION'.                                                                                                                                                                                                                                                                                                                                                                                           | [-0.02565038  0.0092135   0.08270036 ... -0.03528155  0.00391267        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.0010941 ]                                                           |
| openaivec.aio.pandas_ext | _get_openai_client                   | function            | Retrieves the currently registered OpenAI client. If no client is registered, it attempts to create one from environment variables for either OpenAI or Azure OpenAI. If no suitable credentials are found, it raises a ValueError instructing the user to set environment variables or register a client manually.                                                                                                                                  | What happens if no client is registered and no environment variables are set?          | The function raises a ValueError explaining that no API key was found and instructs how to set environment variables or register a client manually.                                                                                                                                                                                                                                                                                                                                      | [-0.04091008  0.03367254  0.09326676 ... -0.04031978  0.0024478         |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01569417]                                                           |
| openaivec.aio.pandas_ext | _extract_value                       | function            | Converts a single Series element into a homogeneous dictionary representation. If the element is None, returns an empty dict. If it is a Pydantic BaseModel, it calls model_dump() to get a dict. If it is already a dict, returns it as is. Otherwise, logs a warning and returns an empty dict.                                                                                                                                                    | Why is this function needed?                                                           | It ensures that values in a Series can be uniformly expanded into DataFrame columns by converting them into dictionaries.                                                                                                                                                                                                                                                                                                                                                                | [ 0.00410907  0.03801276  0.01597459 ...  0.00692925 -0.02591639        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.03034865]                                                           |
| openaivec.aio.pandas_ext | _extract_value                       | function            | Converts a single Series element into a homogeneous dictionary representation. If the element is None, returns an empty dict. If it is a Pydantic BaseModel, it calls model_dump() to get a dict. If it is already a dict, returns it as is. Otherwise, logs a warning and returns an empty dict.                                                                                                                                                    | What happens if the value is not a dict or BaseModel?                                  | A warning is logged, and an empty dictionary is returned to avoid errors during DataFrame expansion.                                                                                                                                                                                                                                                                                                                                                                                     | [-0.02464101  0.03012712  0.01713248 ...  0.01301789 -0.012367          |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0487706 ]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecSeriesAccessor              | class               | A pandas Series accessor registered under the name 'aio' that adds asynchronous OpenAI helper methods to pandas Series objects. It provides methods to generate text responses, compute embeddings, count tokens, and extract structured data from Series elements.                                                                                                                                                                                  | How do you use the responses method?                                                   | Call it asynchronously with instructions and optionally a response_format and batch_size. It returns a Series of responses from the OpenAI model for each element.                                                                                                                                                                                                                                                                                                                       | [-0.02522162  0.01907218  0.06191031 ... -0.00871913 -0.01610143        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00395109]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecSeriesAccessor              | class               | A pandas Series accessor registered under the name 'aio' that adds asynchronous OpenAI helper methods to pandas Series objects. It provides methods to generate text responses, compute embeddings, count tokens, and extract structured data from Series elements.                                                                                                                                                                                  | What does the embeddings method return?                                                | It returns a Series of numpy arrays representing the embedding vectors for each Series element.                                                                                                                                                                                                                                                                                                                                                                                          | [-1.7131567e-02 -1.6577659e-02  4.0593505e-02 ... -1.2364007e-02        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -9.2111848e-05  1.0438519e-02]                                         |
| openaivec.aio.pandas_ext | OpenAIVecSeriesAccessor              | class               | A pandas Series accessor registered under the name 'aio' that adds asynchronous OpenAI helper methods to pandas Series objects. It provides methods to generate text responses, compute embeddings, count tokens, and extract structured data from Series elements.                                                                                                                                                                                  | Is count_tokens asynchronous?                                                          | No, count_tokens is synchronous and returns a Series of token counts per element.                                                                                                                                                                                                                                                                                                                                                                                                        | [-0.01752082  0.00642709  0.00446054 ... -0.00408679 -0.02245291        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.02554071]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecSeriesAccessor              | class               | A pandas Series accessor registered under the name 'aio' that adds asynchronous OpenAI helper methods to pandas Series objects. It provides methods to generate text responses, compute embeddings, count tokens, and extract structured data from Series elements.                                                                                                                                                                                  | What does the extract method do?                                                       | It expands a Series of dicts or Pydantic models into a DataFrame with columns for each key, optionally prefixing columns with the Series name.                                                                                                                                                                                                                                                                                                                                           | [-0.0354525   0.02179999 -0.02859872 ... -0.00533301 -0.05125199        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.0202861 ]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecDataFrameAccessor           | class               | A pandas DataFrame accessor registered under the name 'aio' that adds asynchronous OpenAI helper methods to pandas DataFrame objects. It provides methods to extract nested dict/model columns and generate text responses for each row serialized as JSON.                                                                                                                                                                                          | How does the extract method work?                                                      | It flattens one specified column containing dicts or Pydantic models into separate top-level columns, drops the original column, and preserves the DataFrame index.                                                                                                                                                                                                                                                                                                                      | [-0.04650116  0.02648855 -0.0343481  ... -0.00451817 -0.04732848        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |   0.01841503]                                                           |
| openaivec.aio.pandas_ext | OpenAIVecDataFrameAccessor           | class               | A pandas DataFrame accessor registered under the name 'aio' that adds asynchronous OpenAI helper methods to pandas DataFrame objects. It provides methods to extract nested dict/model columns and generate text responses for each row serialized as JSON.                                                                                                                                                                                          | How do you use the responses method on a DataFrame?                                    | Call it asynchronously with instructions and optionally a response_format and batch_size. It serializes each row to JSON and sends it to the OpenAI model, returning a Series of responses aligned with the DataFrame index.                                                                                                                                                                                                                                                             | [-0.01652777 -0.00087325  0.05201519 ...  0.02238229 -0.00982885        |
|                          |                                      |                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |  -0.00526626]                                                           |